In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
# load datasets
datasets = {}
years = range(2016, 2023)
for year in years:
    print(f"EIE {year} Loading...")
    if int(year) >= 2019:
        file_name = f'Odata{year}File.csv'
    else:
        file_name = f'OpenData{year}.csv'
    try:           
        dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='utf-8')
    except:
        dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='Windows 1251')
    datasets.update({year:dataset})
    print("success")

EIE 2016 Loading...


success
EIE 2017 Loading...
success
EIE 2018 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/3686568105.py:11: DtypeWarning: Columns (119) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='utf-8')


success
EIE 2019 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/3686568105.py:13: DtypeWarning: Columns (100,116,117,118,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='Windows 1251')


success
EIE 2020 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/3686568105.py:13: DtypeWarning: Columns (96,97,98,100,102,103,104,105,116,117,118,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='Windows 1251')


success
EIE 2021 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/3686568105.py:11: DtypeWarning: Columns (117,118,119,121,123,124,125,126,137,138,139,141,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='utf-8')


success
EIE 2022 Loading...
success


In [3]:
for year, dataset in datasets.items():
    #lowercase all columns and add year as attribute
    dataset.columns = [col.lower() for col in dataset.columns]
    dataset['year'] = year

In [4]:
for year, dataset in datasets.items():
    print([col for col in dataset.columns if col.startswith('eo')])

['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']


In [5]:
schools_edrpou = pd.DataFrame(columns=['eoname', 'eoregname', 'eoareaname', 'eotername', 'year'])

In [6]:
for year, dataset in datasets.items():
    schools_edrpou = pd.concat([schools_edrpou, dataset.loc[:, ['eoname', 'eoregname', 'eoareaname', 'eotername', 'year']]], ignore_index=True)

schools_edrpou

,eoname,eoregname,eoareaname,eotername,year
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",Запорізька область,Мелітопольський район,с.Терпіння,2016
1,Красилівська загальноосвітня школа I-III ступе...,Хмельницька область,Красилівський район,м.Красилів,2016
2,NaN,NaN,NaN,NaN,2016
3,Чернівецька спеціалізована школа І-ІІІ ступені...,Чернівецька область,м.Чернівці,Шевченківський район міста,2016
4,NaN,NaN,NaN,NaN,2016
...,...,...,...,...,...
2201113,Заболотцівський ліцей Литовезької сільської ра...,Волинська область,Володимирський район,с.Заболотці,2022
2201114,Заболотцівський ліцей Литовезької сільської ра...,Волинська область,Володимирський район,с.Заболотці,2022
2201115,Заболотцівський ліцей Литовезької сільської ра...,Волинська область,Володимирський район,с.Заболотці,2022
2201116,Заболотцівський ліцей Литовезької сільської ра...,Волинська область,Володимирський район,с.Заболотці,2022


In [7]:
schools_edrpou.drop_duplicates(inplace=True)
# When the paricipant graduated in the previous years 
# the corresponding information about school (everything with 'eo') is NaN
# we drop these rows.
schools_edrpou.dropna(subset=schools_edrpou.columns.difference(['year']), 
                      how = 'all', inplace=True)
schools_edrpou.reset_index(drop=True, inplace=True)
schools_edrpou

,eoname,eoregname,eoareaname,eotername,year
0,"Терпіннівський колегіум ""Джерело"" Мелітопольсь...",Запорізька область,Мелітопольський район,с.Терпіння,2016
1,Красилівська загальноосвітня школа I-III ступе...,Хмельницька область,Красилівський район,м.Красилів,2016
2,Чернівецька спеціалізована школа І-ІІІ ступені...,Чернівецька область,м.Чернівці,Шевченківський район міста,2016
3,Загальноосвітня школа I-III ступенів № 6 Дружк...,Донецька область,Донецька область,м.Дружківка,2016
4,Тернопільська спеціалізована школа І-ІІІ ступе...,Тернопільська область,Тернопільська область,м.Тернопіль,2016
...,...,...,...,...,...
73406,Гульський ліцей Стриївської сільської ради Нов...,Житомирська область,Новоград-Волинський район,с.Гульськ,2022
73407,Киківський ліцей Стриївської сільської ради Но...,Житомирська область,Новоград-Волинський район,с.Кикова,2022
73408,Литовезький ліцей імені Володимира Якобчука Во...,Волинська область,Володимирський район,с.Литовеж,2022
73409,Заболотцівський ліцей Литовезької сільської ра...,Волинська область,Володимирський район,с.Заболотці,2022


In [8]:
locations = pd.read_csv('./location_data/locations.csv', encoding='utf-8', dtype=str)
locations

,regname,areaname,tername,KOATUU_2020,KATOTTG_2023
0,Запорізька область,Мелітопольський район,с.Терпіння,2323085101,UA23080270010078454
1,Хмельницька область,Красилівський район,м.Красилів,6822710100,UA68040210010032567
2,Дніпропетровська область,Петропавлівський район,с.Дмитрівка,1223881501,UA12140170040016918
3,Чернівецька область,м.Чернівці,Шевченківський район міста,7310100000,UA73060610010033137
4,Миколаївська область,Врадіївський район,с.Кумарі,4822383001,UA48080050190079797
...,...,...,...,...,...
29214,США,м.Детройт,м.Детройт,0026010000,OC26010000000000000
29215,Велика Британія,м.Абериствіт,м.Абериствіт,0008030000,OC08030000000000000
29216,США,м.Нью-Йорк,м.Нью-Йорк,0026020000,OC26020000000000000
29217,США,м.Х'юстон,м.Х'юстон,0026040000,OC26040000000000000


In [9]:
schools_edrpou_loc = schools_edrpou.merge(locations[['KOATUU_2020']], left_on=[schools_edrpou['eoregname'],schools_edrpou['eoareaname'], schools_edrpou['eotername']], right_on=[locations['regname'],locations['areaname'], locations['tername']], how='left')

schools_edrpou_loc = schools_edrpou_loc.groupby(['eoname', 'KOATUU_2020'])['year'].agg(lambda x: list(x.unique())).reset_index()

schools_edrpou_loc

,eoname,KOATUU_2020,year
0,Славгородська загальноосвітня школа I-II...,5922386401,"[2016, 2017]"
1,"Івашківський навчально-виховний комплекс ""Заг...",5122582601,[2016]
2,"Грабівський навчально-виховний комплекс ""Зага...",5122581601,[2016]
3,"Кримківська ордена ""Знак Пошани"" ЗОШ І-ІІІ ст...",4825482206,[2016]
4,Лип'янська загальноосвітня школа І-ІІІ ступен...,7125784001,[2016]
...,...,...,...
27505,філія комунального опорного закладу освіти «Чо...,5123184201,[2019]
27506,"філія навчально-виховного комплексу ""Голованів...",3521482801,[2017]
27507,школа І-ІІІ ступенів № 104 імені О.Ольжича Обо...,8038000000,"[2018, 2019, 2020, 2021, 2022]"
27508,школа І-ІІІ ступенів №175 ім.В Марченка Шевчен...,8039100000,"[2020, 2021, 2022]"


In [10]:
schools_edrpou = schools_edrpou_loc[schools_edrpou_loc.KOATUU_2020.notna()]

In [11]:
schools_edrpou[schools_edrpou.KOATUU_2020.str.len() != 10]

,eoname,KOATUU_2020,year


In [12]:
schools_edrpou.shape

(27510, 3)

In [13]:
schools_edrpou['KOATUU_2020_reg'] = schools_edrpou['KOATUU_2020'].str[:2]
schools_edrpou

,eoname,KOATUU_2020,year,KOATUU_2020_reg
0,Славгородська загальноосвітня школа I-II...,5922386401,"[2016, 2017]",59
1,"Івашківський навчально-виховний комплекс ""Заг...",5122582601,[2016],51
2,"Грабівський навчально-виховний комплекс ""Зага...",5122581601,[2016],51
3,"Кримківська ордена ""Знак Пошани"" ЗОШ І-ІІІ ст...",4825482206,[2016],48
4,Лип'янська загальноосвітня школа І-ІІІ ступен...,7125784001,[2016],71
...,...,...,...,...
27505,філія комунального опорного закладу освіти «Чо...,5123184201,[2019],51
27506,"філія навчально-виховного комплексу ""Голованів...",3521482801,[2017],35
27507,школа І-ІІІ ступенів № 104 імені О.Ольжича Обо...,8038000000,"[2018, 2019, 2020, 2021, 2022]",80
27508,школа І-ІІІ ступенів №175 ім.В Марченка Шевчен...,8039100000,"[2020, 2021, 2022]",80


In [14]:
def create_id(dataset, attr, id):
    dataset.loc[:,id] = dataset[attr]

    abbr = {'B':'В', 'E':'Е', 'H':'Н', 'M':'М',
            'T':'Т', 'I':'І', 'O':'О', 'P':'Р',
            'A':'А', 'K':'К', 'X':'Х', 'C':'С',
            'КЗ ':'Комунальний заклад', 'ОНЗ ':'Опорний навчальний заклад ', 
            'ОЗЗСО ':'Опорний заклад загальної середньої освіти ', 'ЗНЗ ':'Загальноосвітній навчальний заклад ',
            'ЗЗСО ':'Заклад загальної середньої освіти ', 'ПЗО ':'Приватний заклад освіти ', 
            'ООЗ ':'Опорний освітній заклад ', 'ЗСО ':'Загальної середньої освіти ', 
            'ЗОШ ':'Загальноосвітня школа ', 'НВК ':'Навчально-виховний комплекс ', 
            'ВСП ':'Відокремлений структурний підрозділ ', 'СЗШ ':'Середня загальноосвітня школа ', 
            'ТОВ ': 'Товариство з обмеженою відповідальністю ', 
            'ОЗ ': 'Опорний заклад '}

    for ab in abbr:
        dataset.loc[:,id] = dataset[id].str.replace(ab, abbr[ab])
        
    dataset.loc[:,id] = dataset[id].str.replace(r'[\s\r\n\t]+', '', regex=True)

    dataset.loc[dataset[id].str.endswith('обл'), id] = dataset.loc[dataset['id'].str.endswith('обл'), id] + '.'
    dataset.loc[:,id] = dataset[id].str.replace('І', 'I')
    dataset.loc[:,id] = dataset[id].str.replace('I-III', '3')
    dataset.loc[:,id] = dataset[id].str.replace('I-II', '2')
    dataset.loc[:,id] = dataset[id].str.lower()

    dataset['id'] = dataset['id'].apply(lambda x: x[:-1])

    dct = {'c':'с', 'e':'е',
           'i':'і', 'o':'о',
           'y':'у', 'p':'р',
           'a':'а', 'k':'к',
           'x': 'х', 
           'села':'с', 'село':'с',  
           'ст.':'ступ', "ступ.":"ступ", 
           "ступеня":"ступ", "ступенів":"ступ", 
           'ім.':'імені',
           'обл.': 'обл', "області":"обл", "область":"обл", 
           "місто":"м", "міста":"м",
           "фаховий":"",
        #    "комунальнийзаклад":"",
        '||||':'3', '|||':'2',
        'і':'', 'ї':'', 'ґ':'г', 'є':'',
        "'":'', "’":'', ",":'', ".":'',"„":"","“": "", "«":'', "»":'', "–":'', "-":'', "*":"", '"':"", ":":"",
        ";": "", "!": "", "?":"",
        "_x000d_":'', '_x000D_':'', "_":'', "(":'', ")":'', "[":'', "]":'', "}":'', "{":'', 'нн':'н', "№":""}

    for symb in dct:
        dataset.loc[:,id] = dataset[id].str.replace(symb, dct[symb])

    dataset.loc[:,id] = dataset[id].str.replace('"', '', regex = True)
    dataset.loc[:,id] = dataset[id].str.replace('комунальнаорганзацяустановазаклад', '')
    

    return dataset


In [15]:
create_id(schools_edrpou, 'eoname', 'id')

,eoname,KOATUU_2020,year,KOATUU_2020_reg,id
0,Славгородська загальноосвітня школа I-II...,5922386401,"[2016, 2017]",59,славгородськазагальноосвтняшкола3ступкраснопль...
1,"Івашківський навчально-виховний комплекс ""Заг...",5122582601,[2016],51,вашквськийнавчальновиховнийкомплексзагальноосв...
2,"Грабівський навчально-виховний комплекс ""Зага...",5122581601,[2016],51,грабвськийнавчальновиховнийкомплексзагальноосв...
3,"Кримківська ордена ""Знак Пошани"" ЗОШ І-ІІІ ст...",4825482206,[2016],48,кримквськаорденазнакпошанизагальноосвтняшкола3...
4,Лип'янська загальноосвітня школа І-ІІІ ступен...,7125784001,[2016],71,липянськазагальноосвтняшкола3ступшполянськорай...
...,...,...,...,...,...
27505,філія комунального опорного закладу освіти «Чо...,5123184201,[2019],51,флякомунальногоопорногозакладуосвтичорнянськаз...
27506,"філія навчально-виховного комплексу ""Голованів...",3521482801,[2017],35,флянавчальновиховногокомплексуголованвськазага...
27507,школа І-ІІІ ступенів № 104 імені О.Ольжича Обо...,8038000000,"[2018, 2019, 2020, 2021, 2022]",80,школа3ступ104меноольжичаоболонськогорайонумкив
27508,школа І-ІІІ ступенів №175 ім.В Марченка Шевчен...,8039100000,"[2020, 2021, 2022]",80,школа3ступ175менвмарченкашевченквськогорайонумкив


# MON data 2023

## Повна назва

In [16]:
schools_data = pd.read_excel('./school_data/Schools_02.08.2023 16-56.xlsx', dtype = str)
schools_data

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,Код ЄДРПОУ,Статус,Тип закладу,Форма власності,Код КОАТУУ,Регіон,...,Телефон,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,(0800)330810,NaN,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)5264358,(044)5262002,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,"(044)234-50-55, (044)235-60-63",(044)234-50-55,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)259-31-60,(044)525-30-31,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)2584242,NaN,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,(046)422-43-40,NaN,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,(0462)687733,NaN,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,(04653)27571,NaN,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,(046)5924644,NaN,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90


In [17]:
schools_data.columns

Index(['Повна назва', 'Код в ЄДЕБО', 'ОУО підтвердив дані', 'Скорочена назва',
       'Код ЄДРПОУ', 'Статус', 'Тип закладу', 'Форма власності', 'Код КОАТУУ',
       'Регіон', 'Населений пункт', 'Місцезнаходження (юридична адреса)',
       'Головний заклад',
       'Найменування органу, до сфери управління якого належить заклад освіти',
       'Телефон', 'Факс', 'Електронна пошта', 'Веб-сайт', 'Керівник',
       'Опорний / Філія', 'Сільський', 'Гірський', 'Інтернат',
       'Ліцензовані обсяги'],
      dtype='object')

In [18]:
schools_data_id = schools_data

In [19]:
create_id(schools_data_id, 'Повна назва', 'id')

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,Код ЄДРПОУ,Статус,Тип закладу,Форма власності,Код КОАТУУ,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,NaN,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN,приватнийзакладосвтикмдш2мкиввдокремленийструк...
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)5262002,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN,гмназя236мкив
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)234-50-55,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN,гмназя87мкив
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,(044)525-30-31,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN,гмназя150мкив
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,NaN,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN,гмназя179мкив
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,"Яблунівський навчально-виховний комплекс ""Зага...",144723,так,NaN,33329891,ліквідовано,навчально-виховний комплекс (об'єднання),Комунальна,7423886502,Чернігівська область,...,NaN,solovejzanna8@gmail.com,NaN,Директор Новік Ольга Іванівна,ні,так,ні,ні,1,яблунвськийнавчальновиховнийкомплексзагальноос...
17331,"Ягіднянська філія опорного закладу ""Количівськ...",145351,так,NaN,43195202,ліквідовано,школа,Комунальна,7425582803,Чернігівська область,...,NaN,yagidneschool@ukr.net,NaN,Завідувач філією Курилець Ніна Михайлівна,Філія,так,ні,ні,NaN,ягднянськафляопорногозакладуколичвськазагально...
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,NaN,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1,ядутинськазагальноосвтняшкола3ступборзнянськом...
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,NaN,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90,янжулвськафля2ступсеменвськогмназ2семенвськомс...


In [20]:
schools_data_id.rename(columns={'Код КОАТУУ':'KOATUU_2020', 'Код ЄДРПОУ':'EDRPOU'}, inplace=True)
schools_data_id.columns

Index(['Повна назва', 'Код в ЄДЕБО', 'ОУО підтвердив дані', 'Скорочена назва',
       'EDRPOU', 'Статус', 'Тип закладу', 'Форма власності', 'KOATUU_2020',
       'Регіон', 'Населений пункт', 'Місцезнаходження (юридична адреса)',
       'Головний заклад',
       'Найменування органу, до сфери управління якого належить заклад освіти',
       'Телефон', 'Факс', 'Електронна пошта', 'Веб-сайт', 'Керівник',
       'Опорний / Філія', 'Сільський', 'Гірський', 'Інтернат',
       'Ліцензовані обсяги', 'id'],
      dtype='object')

In [21]:
schools_data_id[schools_data_id.KOATUU_2020.str.len() != 10]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id


In [22]:
schools_data_id[(schools_data_id.EDRPOU.str.len() < 8)&(schools_data_id.EDRPOU.str.len().notna())]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Факс,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id
3906,Брониківська загальноосвітня школа I-II ступен...,145546,так,NaN,6669916,працює,школа,Комунальна,1824080401,Житомирська область,...,NaN,bronykyzosh@ukr.net,http://bronuku.at.ua/,О.М. Алєксєєва,ні,так,ні,ні,1,брониквськазагальноосвтняшкола2ступновоградвол...
3940,Вепринський навчально-виховний комплекс І ступеню,146135,так,NaN,2206723,працює,навчально-виховний комплекс (об'єднання),Комунальна,1825081101,Житомирська область,...,NaN,veprinnvk@ukr.net,NaN,Г.С. Миненко,ні,так,ні,ні,1,вепринськийнавчальновиховнийкомплексступен
4077,Калинівська загальноосвітня школа I-II ступені...,146872,так,NaN,6669980,працює,школа,Комунальна,1824084802,Житомирська область,...,NaN,kalinivka@i.ua,http://kalin-shkola.at.ua/,І.Й. Жигун,ні,так,ні,ні,1,калинвськазагальноосвтняшкола2ступновоградволи...
4118,Міжнародний приватний навчально-виховний компл...,149486,так,Міжнародний приватний НВК імені Януша Корчака,4068919,працює,навчально-виховний комплекс (об'єднання),Комунальна,1822510100,Житомирська область,...,NaN,grigory.khmelnytsky@gmail.com,NaN,,ні,ні,ні,ні,1,мжнароднийприватнийнавчальновиховнийкомплексме...
4354,Селецька загальноосвітня школа I-II ступенів Н...,148312,так,NaN,6670486,працює,школа,Комунальна,1823787401,Житомирська область,...,NaN,school_selets@i.ua,NaN,І.В. Ніконенко,ні,так,ні,ні,1,селецьказагальноосвтняшкола2ступнародицькогора...
15978,Зюбриська загальноосвітня школа І ступеня Мона...,140731,так,NaN,2363037,працює,школа,Комунальна,7123481701,Черкаська область,...,NaN,Zybr.chkola@gmail.com,NaN,,ні,так,ні,ні,1,зюбриськазагальноосвтняшколаступмонастирищенсь...


In [23]:
schools_data_id['EDRPOU']=schools_data_id['EDRPOU'].apply(lambda x: '0'+x if (isinstance(x, str) and 0<len(x)<8) else x)

In [24]:
schools_data_id = schools_data_id[schools_data_id.EDRPOU.str.len() == 8]

In [25]:
schools_data_id['KOATUU_2020_reg'] = schools_data_id['KOATUU_2020'].str[:2]

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/2930344109.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schools_data_id['KOATUU_2020_reg'] = schools_data_id['KOATUU_2020'].str[:2]


In [26]:
schools_data_id[schools_data_id.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
714,Бершадський заклад загальної середньої освіти ...,136101,так,Бершадський ЗЗСО І-ІІІ ст. №2,23056463,працює,школа,Комунальна,0520410100,Вінницька область,...,bsh-2@ukr.net,http://bershad-school2.edukit.vn.ua/,Директор Ковальський Володимир Михайлович,ні,ні,ні,ні,NaN,бершадськийзакладзагальносередньоосвти3ступ,05
715,Бершадський заклад загальної середньої освіти ...,138581,так,Бершадський ЗЗСО І-ІІІ ступенів №3,23056598,працює,школа,Комунальна,0520410100,Вінницька область,...,school3bershad@ukr.net,http://school3bershad.vn.ua,Директор Шабовта Сергій Олександрович,ні,ні,ні,ні,NaN,бершадськийзакладзагальносередньоосвти3ступ,05
1611,Велимченський ліцей Велимченської сільської ра...,176569,так,Велимченський ліцей,21747814,ліквідовано,ліцей,Комунальна,0724280401,Волинська область,...,NaN,NaN,Директор Павлович Тетяна Володимирівна,ні,ні,ні,ні,NaN,велимченськийлцейвелимченськосльськорадиковель...,07
1612,Велимченський ліцей Велимченської сільської ра...,143856,так,Велимченський ліцей,21747814,працює,ліцей,Комунальна,0724280401,Волинська область,...,velimche__shcola@ukr.net,http://velimcheshkola.at.ua/,Павлович Тетяна Володимирівна,ні,так,ні,ні,NaN,велимченськийлцейвелимченськосльськорадиковель...,07
4728,Данилівський заклад загальної середньої освіти...,176502,так,Данилівський НВК І-ІІІ ступенів,40760974,працює,навчально-виховний комплекс (об'єднання),Комунальна,2125382601,Закарпатська область,...,danylovo@ukr.net,NaN,Директор Гайович Тетяна Василівна,ні,так,ні,ні,171,данилвськийзакладзагальносередньоосвти3ступхус...,21
4729,Данилівський заклад загальної середньої освіти...,148256,так,ДАНИЛІВСЬКИЙ ЗЗСО І-ІІІ СТУПЕНІВ,34440724,реорганізовано,навчально-виховний комплекс (об'єднання),Комунальна,2125382601,Закарпатська область,...,danylovo@ukr.net,NaN,Директор Гайович Тетяна Василівна,ні,так,ні,ні,NaN,данилвськийзакладзагальносередньоосвти3ступхус...,21
5414,Філія Великобілозерського навчально - виховног...,136032,так,Філія Великобілозерського НВК №1 ЗОШ №2,26291070,працює,школа,Комунальна,2321180101,Запорізька область,...,vb-zosch2@meta.ua,NaN,Завідувач філією Маликіна Тетяна Василівна,Філія,так,ні,ні,NaN,флявеликоблозерськогонавчальновиховногокомплек...,23
5416,Філія Великобілозерського навчально- виховного...,136042,так,Філія Великобілозерського НВК №1 ЗОШ №5,26291070,працює,школа,Комунальна,2321180101,Запорізька область,...,vbperv@meta.ua,NaN,Завідувач філією Сиротюк Ольга Василівна,Філія,так,ні,ні,NaN,флявеликоблозерськогонавчальновиховногокомплек...,23
8268,середня загальноосвітня школа №30,148064,так,СЗШ №30,23888560,працює,школа,Комунальна,4610137500,Львівська область,...,school_30@ukr.net,NaN,Директор Михалина Світлана Михайлівна,ні,ні,ні,ні,1,середнязагальноосвтняшкола3,46
8269,Середня загальноосвітня школа №32,147166,так,СЗШ №32,20834306,працює,школа,Комунальна,4610136800,Львівська область,...,school32lv@ukr.net,NaN,Директор Пинянський Микола Зіновійович,ні,ні,ні,ні,960,середнязагальноосвтняшкола3,46


In [27]:
schools_data_id.drop(schools_data_id[schools_data_id.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)].index.values.tolist(), inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/1151126891.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schools_data_id.drop(schools_data_id[schools_data_id.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)].index.values.tolist(), inplace=True)


In [28]:
schools_edrpou['KOATUU_2020_reg'] = schools_edrpou['KOATUU_2020'].str[:2]

In [29]:
schools_ed = schools_edrpou.merge(schools_data_id[['KOATUU_2020_reg', 'EDRPOU', 'id']], on=['id', 'KOATUU_2020_reg'], how='left')
schools_ed.head()

,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU
0,Славгородська загальноосвітня школа I-II...,5922386401,"[2016, 2017]",59,славгородськазагальноосвтняшкола3ступкраснопль...,NaN
1,"Івашківський навчально-виховний комплекс ""Заг...",5122582601,[2016],51,вашквськийнавчальновиховнийкомплексзагальноосв...,NaN
2,"Грабівський навчально-виховний комплекс ""Зага...",5122581601,[2016],51,грабвськийнавчальновиховнийкомплексзагальноосв...,NaN
3,"Кримківська ордена ""Знак Пошани"" ЗОШ І-ІІІ ст...",4825482206,[2016],48,кримквськаорденазнакпошанизагальноосвтняшкола3...,NaN
4,Лип'янська загальноосвітня школа І-ІІІ ступен...,7125784001,[2016],71,липянськазагальноосвтняшкола3ступшполянськорай...,NaN


In [30]:
schools_ed.shape

(27510, 6)

In [31]:
schools_ed[schools_ed['EDRPOU'].isna()].year.value_counts()

year
[2016]                            2768
[2022]                            1542
[2016, 2017]                      1426
[2016, 2017, 2018, 2019, 2020]    1294
[2021, 2022]                      1122
                                  ... 
[2018, 2022]                         1
[2017, 2020, 2021, 2022]             1
[2016, 2019, 2020, 2021, 2022]       1
[2016, 2020, 2021, 2022]             1
[2016, 2018, 2020, 2022]             1
Name: count, Length: 105, dtype: int64

In [32]:
schools_ed.loc[schools_ed['EDRPOU'].isna(), 'EDRPOU'] = ''

In [33]:
schools_ed[schools_ed.EDRPOU != '']

,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU
10,"""Великолазівський ліцей"" Баранинської сільсько...",2124880305,"[2021, 2022]",21,великолазвськийлцейбаранинськосльськорадизакар...,43758931
13,"""Глинівецький ліцей"" Червоненської селищної ра...",1820355602,[2022],18,глинвецькийлцейчервоненськоселищнорадижитомирс...,20408447
14,"""Демино-Олександрівський ліцей Троїцької селищ...",4425455109,"[2019, 2020, 2021]",44,деминоолександрвськийлцейтроцькоселищнорадитро...,20184835
17,"""Комунальна установа ""Гандрабурівський ліцей А...",5120281001,[2021],51,комунальнаустановагандрабурвськийлцейананьвськ...,24778193
18,"""Комунальна установа ""Жеребківський ліцей Анан...",5120282001,[2022],51,комунальнаустановажеребквськийлцейананьвськомс...,24778359
...,...,...,...,...,...,...
27487,опорний заклад загальної середньої освіти Маші...,5323055100,"[2021, 2022]",53,опорнийзакладзагальносередньоосвтимашвськийлце...,23551725
27488,опорний заклад загальної середньої освіти Селе...,5323086601,"[2021, 2022]",53,опорнийзакладзагальносередньоосвтиселещинський...,23551694
27496,середня загальноосвітня школа №67 м. Львова,4610136300,[2021],46,середнязагальноосвтняшкола67мльвов,22350249
27507,школа І-ІІІ ступенів № 104 імені О.Ольжича Обо...,8038000000,"[2018, 2019, 2020, 2021, 2022]",80,школа3ступ104меноольжичаоболонськогорайонумкив,22880852


In [34]:
schools_ed

,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU
0,Славгородська загальноосвітня школа I-II...,5922386401,"[2016, 2017]",59,славгородськазагальноосвтняшкола3ступкраснопль...,
1,"Івашківський навчально-виховний комплекс ""Заг...",5122582601,[2016],51,вашквськийнавчальновиховнийкомплексзагальноосв...,
2,"Грабівський навчально-виховний комплекс ""Зага...",5122581601,[2016],51,грабвськийнавчальновиховнийкомплексзагальноосв...,
3,"Кримківська ордена ""Знак Пошани"" ЗОШ І-ІІІ ст...",4825482206,[2016],48,кримквськаорденазнакпошанизагальноосвтняшкола3...,
4,Лип'янська загальноосвітня школа І-ІІІ ступен...,7125784001,[2016],71,липянськазагальноосвтняшкола3ступшполянськорай...,
...,...,...,...,...,...,...
27505,філія комунального опорного закладу освіти «Чо...,5123184201,[2019],51,флякомунальногоопорногозакладуосвтичорнянськаз...,
27506,"філія навчально-виховного комплексу ""Голованів...",3521482801,[2017],35,флянавчальновиховногокомплексуголованвськазага...,
27507,школа І-ІІІ ступенів № 104 імені О.Ольжича Обо...,8038000000,"[2018, 2019, 2020, 2021, 2022]",80,школа3ступ104меноольжичаоболонськогорайонумкив,22880852
27508,школа І-ІІІ ступенів №175 ім.В Марченка Шевчен...,8039100000,"[2020, 2021, 2022]",80,школа3ступ175менвмарченкашевченквськогорайонумкив,26125940


## Коротка назва

In [35]:
schools_data_id_short = schools_data_id[schools_data_id['Скорочена назва'].notna()]

In [36]:
create_id(schools_data_id_short, 'Скорочена назва', 'id')

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/2540449143.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['id'] = dataset['id'].apply(lambda x: x[:-1])


,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
0,«ПРИВАТНИЙ ЗАКЛАД ОСВІТИ «КМДШ» №2 М. КИЇВ» (В...,150646,так,"ПЗО ""КМДШ"" № 2 м. Київ",43031126,ліквідовано,школа,Приватна,8036100000,КИЇВ,...,kmds2@kmds.kiev.ua,https://www.creativeschool.com.ua/,Директор Фісунова Лідія Олексіївна,ні,ні,ні,ні,NaN,приватнийзакладосвтикмдш2мки,80
1,Гімназія № 236 міста Києва,148438,так,Гімназія № 236,22879470,працює,гімназія,Комунальна,8036100000,КИЇВ,...,school236@ukr.net,http://school-236.at.ua/,Директор Добровольська Оксана Миколаївна,ні,ні,ні,ні,NaN,гмназя23,80
2,Гімназія № 87 міста Києва,148421,так,Гімназія № 87,22881515,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymnasium87@ukr.net,http://school87.ucoz.net/,Директор Гентош Ольга Анатоліївна,ні,ні,ні,ні,NaN,гмназя8,80
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN,гмназя15,80
4,гімназія №179 міста Києва,148440,так,гімназія №179 міста Києва,22879435,працює,гімназія,Комунальна,8036100000,КИЇВ,...,gymn179@hotmail.com,http://179.kiev.ua,Директор Бойко Ірина Львівна,ні,ні,ні,ні,NaN,гмназя179мкив,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17328,"комунальний заклад ""Яблунівський ліцей"" Черніг...",139767,так,"КЗ ""Яблунівський ліцей""",05266240,працює,ліцей,Комунальна,7424189701,Чернігівська область,...,Yablunovka@ukr.net,NaN,Директор Шаповаленко Неоніла Олександрівна,Опорний заклад,так,ні,так,170,комунальнийзакладяблунвськийлцей,74
17329,Яблунівська загальноосвітня школа I-III ступен...,144175,так,Яблунівська ЗОШ І-ІІІ ступенів,33208204,працює,школа,Комунальна,7424189701,Чернігівська область,...,yablunivkask@ukr.net,NaN,Директор Колесник Анатолій Васильович,ні,так,ні,ні,320,яблунвськазагальноосвтняшкола3ступен,74
17332,Ядутинська загальноосвітня школа І-ІІІ ступені...,135163,так,Ядутинська ЗОШ І-ІІІ ст.,33365840,працює,школа,Комунальна,7420889601,Чернігівська область,...,yaduti@ukr.net,NaN,Шостак Ірина Володимирівна,ні,так,ні,ні,1,ядутинськазагальноосвтняшкола3ст,74
17333,Янжулівська філія І-ІІ ступенів Семенівської г...,134967,так,Янжулівська філія І-ІІ ст.,25955585,працює,школа,Комунальна,7424781701,Чернігівська область,...,NaN,NaN,Завідувач філією Подоляк Ірина Миколаївна,Філія,так,ні,ні,90,янжулвськафля2ст,74


In [37]:
schools_data_id_short[schools_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)]

,Повна назва,Код в ЄДЕБО,ОУО підтвердив дані,Скорочена назва,EDRPOU,Статус,Тип закладу,Форма власності,KOATUU_2020,Регіон,...,Електронна пошта,Веб-сайт,Керівник,Опорний / Філія,Сільський,Гірський,Інтернат,Ліцензовані обсяги,id,KOATUU_2020_reg
3,Гімназія №150 міста Києва,148431,так,Гімназія № 150,22879429,працює,гімназія,Комунальна,8036100000,КИЇВ,...,150shkola@ukr.net,http://school150.klasna.com/uk,Директор Гришко Юрій Юрійович,ні,ні,ні,ні,NaN,гмназя15,80
28,Спеціалізована школа І-ІІІ ступенів з поглибле...,148398,так,спеціалізована школа,22877005,працює,спеціалізована школа,Комунальна,8036100000,КИЇВ,...,sch15@i.ua,school15.kyiv.ua,Директор Литвин Валерій Олексійович,ні,ні,ні,ні,NaN,спецалзованашкол,80
33,Спеціалізована школа І-ІІІ ступенів з поглибле...,148433,так,СШ №130,22879398,працює,спеціалізована школа,Комунальна,8036100000,КИЇВ,...,school130.kiev@gmail.com,http://school130.org.ua/,Директор Кузьменко Наталія Іванівна,ні,ні,ні,ні,NaN,сш13,80
37,Спеціалізована школа І-ІІІ ступенів з поглибле...,148447,так,СШ №110,22879352,працює,спеціалізована школа,Комунальна,8036100000,КИЇВ,...,shkola110@i.ua,http://school-110.com,Директор Дубовик Лариса Ігорівна,ні,ні,ні,ні,NaN,сш11,80
39,спеціальна школа №11 міста Києва,148446,так,спеціальна школа №11,22879553,працює,спеціальна школа,Комунальна,8036100000,КИЇВ,...,school-internat11@ukr.net,https://school-internat11.wixsite.com/mysite,Директор Кулеша Ірина Андріївна,ні,ні,ні,ні,NaN,спецальнашкола1,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17244,Семенівський заклад загальної середньої освіти...,138323,так,Семенівський ЗСО І-ІІІ ступенів №1,25955556,працює,школа,Комунальна,7424710100,Чернігівська область,...,semenivka.1@meta.ua,http://semenivka-school1.edukit.cn.ua/,Директор Лупан Оксана Михайлівна,ні,ні,ні,ні,570,семенвськийзагальносередньоосвти3ступ,74
17245,Семенівський заклад загальної середньої освіти...,138311,так,Семенівський ЗСО І-ІІІ ст. №3,25955562,працює,школа,Комунальна,7424710100,Чернігівська область,...,semzosh3@meta.ua,NaN,Директор Лозовська Ліана Миколаївна,ні,ні,ні,ні,330,семенвськийзагальносередньоосвти3ступ,74
17261,Сновський ліцей № 1 Сновської міської ради Кор...,146893,так,Сновський ліцей № 1,21402116,працює,ліцей,Комунальна,7425810100,Чернігівська область,...,snovsk_shkola_1@ukr.net,http://schors-school1.edukit.cn.ua,Директор Єрмоленко Алла Олександрівна,Опорний заклад,ні,ні,ні,NaN,сновськийлцей,74
17262,Сновський ліцей № 2 Сновської міської ради Кор...,147020,так,Сновський ліцей № 2,21402033,працює,ліцей,Комунальна,7425810100,Чернігівська область,...,snovskzosh2@ukr.net,NaN,Директор Росик Світлана Миколаївна,ні,ні,ні,ні,NaN,сновськийлцей,74


In [38]:
schools_data_id_short.drop(schools_data_id_short[schools_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)].index.values.tolist(), inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/2976488812.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schools_data_id_short.drop(schools_data_id_short[schools_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)].index.values.tolist(), inplace=True)


In [39]:
def merging(df1, df2, lst_on):
    output = df1.merge(df2, on=lst_on, how='left')
    output.loc[output['EDRPOU_y'].isna(), 'EDRPOU_y'] = ''
    output.loc[output['EDRPOU_x'] == '', 'EDRPOU_x'] = output.loc[output['EDRPOU_x'] == '', 'EDRPOU_y']
    output = output.drop(columns=['EDRPOU_y'], axis=1)
    output = output.rename(columns = {'EDRPOU_x':'EDRPOU'})
    print('Percentage matched:',round(output[output['EDRPOU'] != ''].shape[0]/output.shape[0]*100,1), '%')
    return output

In [40]:
schools_ed = merging(schools_ed, schools_data_id_short[['KOATUU_2020_reg', 'EDRPOU', 'id']], ['id', 'KOATUU_2020_reg'])

Percentage matched: 22.2 %


# Finance 2019

In [41]:
finance_data_2019 =  pd.read_excel('/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/NY fairness/NeurIPS/ZNO-Dataset/notebooks/school_level/fin_data/ZZSO-2019.xlsx', header = 1, dtype='str')
finance_data_2019

,№ Області,Область,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на функціонування закладу (загальний фонд), тис.грн.\n2019 рік (касові видатки)","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (план)","Видатки на 1 учня, 2019, тис. грн.**","Видатки на 1 клас, тис. грн.** 2019 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,ЄДРПОУ,Примітка
0,2,Вінницька,місто,1,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,...,9915.2,11123,18.395547309833024,495.76000000000005,ні,2202100000,комунальна,8173,26244113,NaN
1,2,Вінницька,місто,2,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,...,13692.2,13758.8,16.300238095238097,456.4066666666667,ні,2202100000,комунальна,8174,26244107,NaN
2,2,Вінницька,місто,3,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,...,16184.7,17254.2,16.49816513761468,449.57500000000005,ні,2202100000,комунальна,8175,26244136,NaN
3,2,Вінницька,місто,4,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,...,14321.9,14598.6,16.770374707259954,447.559375,ні,2202100000,комунальна,8176,26244099,NaN
4,2,Вінницька,місто,5,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,...,15061,16011.5,17.431712962962962,456.3939393939394,ні,2202100000,комунальна,8177,23064971,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14170,26,місто Київ,місто,14171,місто Київ,Школа І ступеня № 326,I,NaN,15,12,...,6642.512,7394.666,39.0736,1107.0853333333332,ні,26000000000,комунальна,23601,33100952,NaN
14171,26,місто Київ,місто,14172,місто Київ,Школа І ступеня №333 Дарницького району м.Києва,I,NaN,51.89,27.5,...,10285.5,13232.399,20.0888671875,642.84375,ні,26000000000,комунальна,23680,41659088,NaN
14172,26,місто Київ,місто,14173,місто Київ,Початкова школа №332 Дарницького району м. Києва,I,NaN,80.11,35,...,14547.4,21722.249,15.280882352941177,538.7925925925925,ні,26000000000,комунальна,23730,42384345,NaN
14173,26,місто Київ,місто,14174,місто Київ,Початкова школа № 334 Дарницького району м. Києва,I,NaN,36.22,26,...,6973.8,13217.047,19.86837606837607,633.9818181818182,ні,26000000000,комунальна,23731,42384408,NaN


In [42]:
finance_data_2019.columns

Index(['№ Області', 'Область', 'Тип школи: міська / сільська', '№№',
       'Назва тер. Одиниці', 'Найменування закладу',
       'Ступінь закладу ( I, I-II, I-III)',
       'Загальна площа приміщень школи ( без господарських будівель) (кв.м.)',
       'Кількість штатних працівників педагогічного персоналу  разом',
       'Кількість штатних працівників непедагогічного персоналу  разом',
       'Кількість штатних працівників педагогічного персоналу  пенсійного віку',
       'Питома вага непедагогічного персоналу у загальній кількості працівників, %**',
       'Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %**',
       'Кількість учнів разом',
       'Разом кількість учнів, та дітей, які здобувають дошкільну освіту',
       'Кількість класів разом',
       'Фактична наповнюваність класів, учнів на клас**',
       'Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя**',
       'Видатки на функціонування закладу 

In [43]:
create_id(finance_data_2019, 'Найменування закладу', 'id')

,№ Області,Область,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (план)","Видатки на 1 учня, 2019, тис. грн.**","Видатки на 1 клас, тис. грн.** 2019 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,ЄДРПОУ,Примітка,id
0,2,Вінницька,місто,1,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,...,11123,18.395547309833024,495.76000000000005,ні,2202100000,комунальна,8173,26244113,NaN,загальноосвтняшкола3ступ1комунальновласносттер...
1,2,Вінницька,місто,2,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,...,13758.8,16.300238095238097,456.4066666666667,ні,2202100000,комунальна,8174,26244107,NaN,навчальновиховнийкомплексзагальноосвтняшкола3с...
2,2,Вінницька,місто,3,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,...,17254.2,16.49816513761468,449.57500000000005,ні,2202100000,комунальна,8175,26244136,NaN,загальноосвтняшкола3ступ3комунальновласносттер...
3,2,Вінницька,місто,4,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,...,14598.6,16.770374707259954,447.559375,ні,2202100000,комунальна,8176,26244099,NaN,загальноосвтняшкола3ступ4комунальновластностте...
4,2,Вінницька,місто,5,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,...,16011.5,17.431712962962962,456.3939393939394,ні,2202100000,комунальна,8177,23064971,NaN,навчальновиховнийкомплексзагальноосвтняшкола3с...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14170,26,місто Київ,місто,14171,місто Київ,Школа І ступеня № 326,I,NaN,15,12,...,7394.666,39.0736,1107.0853333333332,ні,26000000000,комунальна,23601,33100952,NaN,школаступ32
14171,26,місто Київ,місто,14172,місто Київ,Школа І ступеня №333 Дарницького району м.Києва,I,NaN,51.89,27.5,...,13232.399,20.0888671875,642.84375,ні,26000000000,комунальна,23680,41659088,NaN,школаступ333дарницькогорайонумкив
14172,26,місто Київ,місто,14173,місто Київ,Початкова школа №332 Дарницького району м. Києва,I,NaN,80.11,35,...,21722.249,15.280882352941177,538.7925925925925,ні,26000000000,комунальна,23730,42384345,NaN,початковашкола332дарницькогорайонумкив
14173,26,місто Київ,місто,14174,місто Київ,Початкова школа № 334 Дарницького району м. Києва,I,NaN,36.22,26,...,13217.047,19.86837606837607,633.9818181818182,ні,26000000000,комунальна,23731,42384408,NaN,початковашкола334дарницькогорайонумкив


In [44]:
finance_data_2019.rename(columns={'ЄДРПОУ':'EDRPOU'}, inplace=True)

In [45]:
finance_data_2019.columns

Index(['№ Області', 'Область', 'Тип школи: міська / сільська', '№№',
       'Назва тер. Одиниці', 'Найменування закладу',
       'Ступінь закладу ( I, I-II, I-III)',
       'Загальна площа приміщень школи ( без господарських будівель) (кв.м.)',
       'Кількість штатних працівників педагогічного персоналу  разом',
       'Кількість штатних працівників непедагогічного персоналу  разом',
       'Кількість штатних працівників педагогічного персоналу  пенсійного віку',
       'Питома вага непедагогічного персоналу у загальній кількості працівників, %**',
       'Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %**',
       'Кількість учнів разом',
       'Разом кількість учнів, та дітей, які здобувають дошкільну освіту',
       'Кількість класів разом',
       'Фактична наповнюваність класів, учнів на клас**',
       'Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя**',
       'Видатки на функціонування закладу 

In [46]:
finance_data_2019[(finance_data_2019.EDRPOU.str.len() < 8)&(finance_data_2019.EDRPOU.str.len().notna())]

,№ Області,Область,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (план)","Видатки на 1 учня, 2019, тис. грн.**","Видатки на 1 клас, тис. грн.** 2019 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,EDRPOU,Примітка,id
52,2,Вінницька,село,53,Барський район,Каришківська загальноосвітня школа І-ІІІ ступенів,I-III,NaN,22,17,...,4541.4,45.751724137931035,361.8545454545455,ні,2301200000,комунальна,6927,2141288,NaN,каришквськазагальноосвтняшкола3ступен
53,2,Вінницька,село,54,Барський район,"Марянівський навчально-виховний комплекс ""ДНЗ-...",I,NaN,7,10,...,1391.1,45.48,1591.8,ні,2301200000,комунальна,22767,0,NaN,марянвськийнавчальновиховнийкомплексднззошступен
224,2,Вінницька,село,225,Крижопільський район,"Навчально- виховний комплекс ""Загальноосвітній...",I-III,NaN,17,13,...,4221.4,55.067647058823525,468.075,ні,2309200000,комунальна,11377,2141360,NaN,навчальновиховнийкомплексзагальноосвтнйнавчаль...
236,2,Вінницька,село,237,Липовецький район,Зозівська загальноосвітня школа І-ІІІ ст. Липо...,I-III,NaN,22,18,...,5075.6,27.579310344827586,479.88,ні,2310200000,комунальна,11395,2141377,NaN,зозвськазагальноосвтняшкола3ступлиповецькорайо...
299,2,Вінницька,село,300,Могилів-Подільський район,Григорівська загальноосвітня школа 1 ступеня,I,NaN,3,2,...,712.694,31.189058823529415,176.73800000000003,ні,2312200000,комунальна,7183,4326572,NaN,григорвськазагальноосвтняшкола1ступен
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13880,26,місто Київ,місто,13881,місто Київ,Школа І-ІІІ ступенів № 282 Деснянського району...,I-III,NaN,67,28,...,22945.9,23.19441674762407,686.4097706249999,ні,26000000000,комунальна,22956,1488067,NaN,школа3ступ282деснянськогорайонумкив
13881,26,місто Київ,місто,13882,місто Київ,Гімназія №283 ІІ-ІІІ ступенів Деснянського рай...,II-III,NaN,71,33,...,26427.3,26.567503649635032,772.0677575757575,ні,26000000000,комунальна,22957,1488937,NaN,гмназя2833ступдеснянськогорайонумкив
13882,26,місто Київ,місто,13883,місто Київ,Школа І-ІІІ ступенів № 292 імені гетьмана Укра...,I-III,NaN,57,27,...,21674.1,26.184724164490863,742.8703225925926,ні,26000000000,комунальна,22958,1488943,NaN,школа3ступ292менгетьманаукраниванамазепидеснян...
13883,26,місто Київ,місто,13884,місто Київ,"Комунальний заклад ""Навчально-виховний комплек...",I-III,NaN,65,39,...,28046.9,25.45172706592386,721.3555276315789,ні,26000000000,комунальна,22959,1489291,NaN,комунальнийзакладнавчальновиховнийкомплексспец...


In [47]:
finance_data_2019 ['EDRPOU']=finance_data_2019['EDRPOU'].apply(lambda x: '0'+x if (isinstance(x, str) and 0<len(x)<8) else x)

In [48]:
finance_data_2019 ['EDRPOU']=finance_data_2019['EDRPOU'].apply(lambda x: '0'+x if (isinstance(x, str) and 0<len(x)<8) else x)

In [49]:
finance_data_2019 = finance_data_2019[finance_data_2019.EDRPOU.str.len() == 8]

In [50]:
locations['KOATUU_2020_reg'] = locations['KOATUU_2020'].str[:2]
locations

,regname,areaname,tername,KOATUU_2020,KATOTTG_2023,KOATUU_2020_reg
0,Запорізька область,Мелітопольський район,с.Терпіння,2323085101,UA23080270010078454,23
1,Хмельницька область,Красилівський район,м.Красилів,6822710100,UA68040210010032567,68
2,Дніпропетровська область,Петропавлівський район,с.Дмитрівка,1223881501,UA12140170040016918,12
3,Чернівецька область,м.Чернівці,Шевченківський район міста,7310100000,UA73060610010033137,73
4,Миколаївська область,Врадіївський район,с.Кумарі,4822383001,UA48080050190079797,48
...,...,...,...,...,...,...
29214,США,м.Детройт,м.Детройт,0026010000,OC26010000000000000,00
29215,Велика Британія,м.Абериствіт,м.Абериствіт,0008030000,OC08030000000000000,00
29216,США,м.Нью-Йорк,м.Нью-Йорк,0026020000,OC26020000000000000,00
29217,США,м.Х'юстон,м.Х'юстон,0026040000,OC26040000000000000,00


In [51]:
locations.regname.unique()

array(['Запорізька область', 'Хмельницька область',
       'Дніпропетровська область', 'Чернівецька область',
       'Миколаївська область', 'Донецька область',
       'Тернопільська область', 'Кіровоградська область',
       'Одеська область', 'Львівська область',
       'Івано-Франківська область', 'Харківська область',
       'Черкаська область', 'Волинська область', 'Полтавська область',
       'Рівненська область', 'Вінницька область', 'Чернігівська область',
       'Київська область', 'Херсонська область', 'Житомирська область',
       'Закарпатська область', 'м.Київ', 'Сумська область',
       'Луганська область', 'Інші країни', 'Польща', 'Чехія', 'Литва',
       'Фінляндія', 'Німеччина', 'Італія', 'Нідерланди',
       'Велика Британія', 'Ірландія', 'Франція', 'Болгарія', 'Іспанія',
       'Швеція', 'Швейцарія', 'Австрія', 'Угорщина', 'Хорватія', 'Канада',
       'Румунія', 'Норвегія', 'Латвія', 'Бельгія', 'Естонія', 'Молдова',
       'Данія', 'Грузія', 'Словаччина', 'Туреччина'

In [52]:
set(finance_data_2019['Область'].unique())-set(locations.regname.unique())

{'Івано-Франківська',
 'Волинська',
 'Вінницька',
 'Дніпропетровська',
 'Донецька',
 'Житомирська',
 'Закарпатська',
 'Запорізька',
 'Київська',
 'Кіровоградська',
 'Луганська',
 'Львівська',
 'Миколаївська',
 'Одеська',
 'Полтавська',
 'Рівненська',
 'Сумська',
 'Тернопільська',
 'Харківська',
 'Херсонська',
 'Хмельницька',
 'Черкаська',
 'Чернівецька',
 'Чернігівська',
 'місто Київ'}

In [53]:
finance_data_2019['Область'] = finance_data_2019['Область'].apply(lambda x: 'м.Київ' if x == 'місто Київ' else x+' область')
finance_data_2019

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/3800352842.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finance_data_2019['Область'] = finance_data_2019['Область'].apply(lambda x: 'м.Київ' if x == 'місто Київ' else x+' область')


,№ Області,Область,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (план)","Видатки на 1 учня, 2019, тис. грн.**","Видатки на 1 клас, тис. грн.** 2019 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,EDRPOU,Примітка,id
0,2,Вінницька область,місто,1,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,...,11123,18.395547309833024,495.76000000000005,ні,2202100000,комунальна,8173,26244113,NaN,загальноосвтняшкола3ступ1комунальновласносттер...
1,2,Вінницька область,місто,2,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,...,13758.8,16.300238095238097,456.4066666666667,ні,2202100000,комунальна,8174,26244107,NaN,навчальновиховнийкомплексзагальноосвтняшкола3с...
2,2,Вінницька область,місто,3,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,...,17254.2,16.49816513761468,449.57500000000005,ні,2202100000,комунальна,8175,26244136,NaN,загальноосвтняшкола3ступ3комунальновласносттер...
3,2,Вінницька область,місто,4,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,...,14598.6,16.770374707259954,447.559375,ні,2202100000,комунальна,8176,26244099,NaN,загальноосвтняшкола3ступ4комунальновластностте...
4,2,Вінницька область,місто,5,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,...,16011.5,17.431712962962962,456.3939393939394,ні,2202100000,комунальна,8177,23064971,NaN,навчальновиховнийкомплексзагальноосвтняшкола3с...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14170,26,м.Київ,місто,14171,місто Київ,Школа І ступеня № 326,I,NaN,15,12,...,7394.666,39.0736,1107.0853333333332,ні,26000000000,комунальна,23601,33100952,NaN,школаступ32
14171,26,м.Київ,місто,14172,місто Київ,Школа І ступеня №333 Дарницького району м.Києва,I,NaN,51.89,27.5,...,13232.399,20.0888671875,642.84375,ні,26000000000,комунальна,23680,41659088,NaN,школаступ333дарницькогорайонумкив
14172,26,м.Київ,місто,14173,місто Київ,Початкова школа №332 Дарницького району м. Києва,I,NaN,80.11,35,...,21722.249,15.280882352941177,538.7925925925925,ні,26000000000,комунальна,23730,42384345,NaN,початковашкола332дарницькогорайонумкив
14173,26,м.Київ,місто,14174,місто Київ,Початкова школа № 334 Дарницького району м. Києва,I,NaN,36.22,26,...,13217.047,19.86837606837607,633.9818181818182,ні,26000000000,комунальна,23731,42384408,NaN,початковашкола334дарницькогорайонумкив


In [54]:
finance_data_2019.rename(columns={'Область':'regname'}, inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/1983970288.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finance_data_2019.rename(columns={'Область':'regname'}, inplace=True)


In [55]:
finance_data_2019

,№ Області,regname,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (план)","Видатки на 1 учня, 2019, тис. грн.**","Видатки на 1 клас, тис. грн.** 2019 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,EDRPOU,Примітка,id
0,2,Вінницька область,місто,1,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,...,11123,18.395547309833024,495.76000000000005,ні,2202100000,комунальна,8173,26244113,NaN,загальноосвтняшкола3ступ1комунальновласносттер...
1,2,Вінницька область,місто,2,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,...,13758.8,16.300238095238097,456.4066666666667,ні,2202100000,комунальна,8174,26244107,NaN,навчальновиховнийкомплексзагальноосвтняшкола3с...
2,2,Вінницька область,місто,3,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,...,17254.2,16.49816513761468,449.57500000000005,ні,2202100000,комунальна,8175,26244136,NaN,загальноосвтняшкола3ступ3комунальновласносттер...
3,2,Вінницька область,місто,4,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,...,14598.6,16.770374707259954,447.559375,ні,2202100000,комунальна,8176,26244099,NaN,загальноосвтняшкола3ступ4комунальновластностте...
4,2,Вінницька область,місто,5,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,...,16011.5,17.431712962962962,456.3939393939394,ні,2202100000,комунальна,8177,23064971,NaN,навчальновиховнийкомплексзагальноосвтняшкола3с...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14170,26,м.Київ,місто,14171,місто Київ,Школа І ступеня № 326,I,NaN,15,12,...,7394.666,39.0736,1107.0853333333332,ні,26000000000,комунальна,23601,33100952,NaN,школаступ32
14171,26,м.Київ,місто,14172,місто Київ,Школа І ступеня №333 Дарницького району м.Києва,I,NaN,51.89,27.5,...,13232.399,20.0888671875,642.84375,ні,26000000000,комунальна,23680,41659088,NaN,школаступ333дарницькогорайонумкив
14172,26,м.Київ,місто,14173,місто Київ,Початкова школа №332 Дарницького району м. Києва,I,NaN,80.11,35,...,21722.249,15.280882352941177,538.7925925925925,ні,26000000000,комунальна,23730,42384345,NaN,початковашкола332дарницькогорайонумкив
14173,26,м.Київ,місто,14174,місто Київ,Початкова школа № 334 Дарницького району м. Києва,I,NaN,36.22,26,...,13217.047,19.86837606837607,633.9818181818182,ні,26000000000,комунальна,23731,42384408,NaN,початковашкола334дарницькогорайонумкив


In [56]:
loc = locations[['regname', 'KOATUU_2020_reg']]
loc.drop_duplicates(inplace= True)
loc.reset_index(drop=True, inplace=True)
loc

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/3378581834.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc.drop_duplicates(inplace= True)


,regname,KOATUU_2020_reg
0,Запорізька область,23
1,Хмельницька область,68
2,Дніпропетровська область,12
3,Чернівецька область,73
4,Миколаївська область,48
5,Донецька область,14
6,Тернопільська область,61
7,Кіровоградська область,35
8,Одеська область,51
9,Львівська область,46


In [57]:
finance_data_2019 = finance_data_2019.merge(loc[['regname', 'KOATUU_2020_reg']], on=['regname'], how='left')
finance_data_2019.head()

,№ Області,regname,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на 1 учня, 2019, тис. грн.**","Видатки на 1 клас, тис. грн.** 2019 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,EDRPOU,Примітка,id,KOATUU_2020_reg
0,2,Вінницька область,місто,1,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №1 комуна...,I-III,NaN,44,20,...,18.395547309833024,495.76000000000005,ні,2202100000,комунальна,8173,26244113,NaN,загальноосвтняшкола3ступ1комунальновласносттер...,05
1,2,Вінницька область,місто,2,Жмеринська міська ОТГ,"Навчально-виховний комплекс ""Загальноосвітня ш...",I-III,NaN,59,18,...,16.300238095238097,456.4066666666667,ні,2202100000,комунальна,8174,26244107,NaN,навчальновиховнийкомплексзагальноосвтняшкола3с...,05
2,2,Вінницька область,місто,3,Жмеринська міська ОТГ,Загальноосвітня школа І-ІІІ ступенів №3 комуна...,I-III,NaN,82,23,...,16.49816513761468,449.57500000000005,ні,2202100000,комунальна,8175,26244136,NaN,загальноосвтняшкола3ступ3комунальновласносттер...,05
3,2,Вінницька область,місто,4,Жмеринська міська ОТГ,Загальноосвітня школа I-III ступенів №4 комуна...,I-III,NaN,61,22,...,16.770374707259954,447.559375,ні,2202100000,комунальна,8176,26244099,NaN,загальноосвтняшкола3ступ4комунальновластностте...,05
4,2,Вінницька область,місто,5,Жмеринська міська ОТГ,"Навчально - виховний комплекс ""Загальноосвітня...",I-III,NaN,70,23,...,17.431712962962962,456.3939393939394,ні,2202100000,комунальна,8177,23064971,NaN,навчальновиховнийкомплексзагальноосвтняшкола3с...,05


In [58]:
finance_data_2019.shape

(14170, 30)

In [59]:
finance_data_2019[finance_data_2019.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)]

,№ Області,regname,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на 1 учня, 2019, тис. грн.**","Видатки на 1 клас, тис. грн.** 2019 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,EDRPOU,Примітка,id,KOATUU_2020_reg
56,2,Вінницька область,місто,58,Бершадський район,Бершадська загальноосвітня школа І-ІІІ ступені...,I-III,NaN,30,14,...,27.630931623931623,340.29673684210525,ні,2302200000,комунальна,6110,23056463,NaN,бершадськазагальноосвтняшкола3ступ,05
62,2,Вінницька область,місто,64,Бершадський район,Бершадська загальноосвітня школа І-ІІІ ступені...,I-III,NaN,51,29,...,18.986766355140187,458.7447741935484,ні,2302200000,комунальна,6124,23056479,NaN,бершадськазагальноосвтняшкола3ступ,05
72,2,Вінницька область,місто,74,Бершадський район,Бершадська загальноосвітня школа І-ІІІ ступені...,I-III,NaN,54,29,...,21.713166666666666,482.5148148148148,ні,2302200000,комунальна,6152,23056598,NaN,бершадськазагальноосвтняшкола3ступ,05
726,2,Вінницька область,місто,728,Літинська селищна ОТГ,Літинська середня загальноосвітня школа І-ІІІ ...,I-III,NaN,53,22,...,18.371435483870968,438.088076923077,ні,2541000000,комунальна,7981,25497740,Літинська ОТГ надає для фінансування районному...,лтинськасереднязагальноосвтняшкола3ступ,05
728,2,Вінницька область,місто,730,Літинська селищна ОТГ,Літинська середня загальноосвітня школаІ-ІІІст...,I-III,NaN,51,23,...,19.005659777424484,459.79076923076923,ні,2541000000,комунальна,8003,02141383,Літинська ОТГ надає для фінансування районному...,лтинськасереднязагальноосвтняшкола3ступ,05
1717,4,Дніпропетровська область,місто,1719,Місто Першотравенськ,Першотравенська загальноосвітня школа І-ІІІ ст...,I-III,NaN,40,14,...,17.772629213483146,395.441,ні,4211100000,комунальна,2722,26329192,NaN,першотравенськазагальноосвтняшкола3ступ,12
1718,4,Дніпропетровська область,місто,1720,Місто Першотравенськ,Першотравенська загальноосвітня школа І-ІІІ ст...,I-III,NaN,58,21,...,15.39304347826087,389.444,ні,4211100000,комунальна,2732,19433824,NaN,першотравенськазагальноосвтняшкола3ступ,12
1719,4,Дніпропетровська область,місто,1721,Місто Першотравенськ,Першотравенська загальноосвітня школа І-ІІІ ст...,I-III,NaN,38,17,...,16.359375,392.625,ні,4211100000,комунальна,2727,19433836,NaN,першотравенськазагальноосвтняшкола3ступ,12
1720,4,Дніпропетровська область,місто,1722,Місто Першотравенськ,Першотравенська загальноосвітня школа І-ІІІ ст...,I-III,NaN,36,15,...,16.635441767068276,394.4976190476191,ні,4211100000,комунальна,3272,19433842,NaN,першотравенськазагальноосвтняшкола3ступ,12
1721,4,Дніпропетровська область,місто,1723,Місто Першотравенськ,Першотравенська загальноосвітня школа І-ІІІ ст...,I-III,NaN,57,23,...,14.642758201701094,376.5934375,ні,4211100000,комунальна,2751,26329200,NaN,першотравенськазагальноосвтняшкола3ступ,12


In [60]:
finance_data_2019.drop(finance_data_2019[finance_data_2019.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)].index.values.tolist()  , inplace=True)

In [61]:
schools_ed['KOATUU_2020_reg'] = schools_ed['KOATUU_2020'].str[:2]
schools_ed

,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU
0,Славгородська загальноосвітня школа I-II...,5922386401,"[2016, 2017]",59,славгородськазагальноосвтняшкола3ступкраснопль...,
1,"Івашківський навчально-виховний комплекс ""Заг...",5122582601,[2016],51,вашквськийнавчальновиховнийкомплексзагальноосв...,
2,"Грабівський навчально-виховний комплекс ""Зага...",5122581601,[2016],51,грабвськийнавчальновиховнийкомплексзагальноосв...,
3,"Кримківська ордена ""Знак Пошани"" ЗОШ І-ІІІ ст...",4825482206,[2016],48,кримквськаорденазнакпошанизагальноосвтняшкола3...,
4,Лип'янська загальноосвітня школа І-ІІІ ступен...,7125784001,[2016],71,липянськазагальноосвтняшкола3ступшполянськорай...,
...,...,...,...,...,...,...
27505,філія комунального опорного закладу освіти «Чо...,5123184201,[2019],51,флякомунальногоопорногозакладуосвтичорнянськаз...,
27506,"філія навчально-виховного комплексу ""Голованів...",3521482801,[2017],35,флянавчальновиховногокомплексуголованвськазага...,
27507,школа І-ІІІ ступенів № 104 імені О.Ольжича Обо...,8038000000,"[2018, 2019, 2020, 2021, 2022]",80,школа3ступ104меноольжичаоболонськогорайонумкив,22880852
27508,школа І-ІІІ ступенів №175 ім.В Марченка Шевчен...,8039100000,"[2020, 2021, 2022]",80,школа3ступ175менвмарченкашевченквськогорайонумкив,26125940


In [62]:
schools_ed_fin = merging(schools_ed, finance_data_2019[['KOATUU_2020_reg', 'id', 'EDRPOU']], ['KOATUU_2020_reg', 'id'])
schools_ed_fin

Percentage matched: 46.3 %


,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU
0,Славгородська загальноосвітня школа I-II...,5922386401,"[2016, 2017]",59,славгородськазагальноосвтняшкола3ступкраснопль...,32758871
1,"Івашківський навчально-виховний комплекс ""Заг...",5122582601,[2016],51,вашквськийнавчальновиховнийкомплексзагальноосв...,
2,"Грабівський навчально-виховний комплекс ""Зага...",5122581601,[2016],51,грабвськийнавчальновиховнийкомплексзагальноосв...,
3,"Кримківська ордена ""Знак Пошани"" ЗОШ І-ІІІ ст...",4825482206,[2016],48,кримквськаорденазнакпошанизагальноосвтняшкола3...,
4,Лип'янська загальноосвітня школа І-ІІІ ступен...,7125784001,[2016],71,липянськазагальноосвтняшкола3ступшполянськорай...,
...,...,...,...,...,...,...
27505,філія комунального опорного закладу освіти «Чо...,5123184201,[2019],51,флякомунальногоопорногозакладуосвтичорнянськаз...,
27506,"філія навчально-виховного комплексу ""Голованів...",3521482801,[2017],35,флянавчальновиховногокомплексуголованвськазага...,
27507,школа І-ІІІ ступенів № 104 імені О.Ольжича Обо...,8038000000,"[2018, 2019, 2020, 2021, 2022]",80,школа3ступ104меноольжичаоболонськогорайонумкив,22880852
27508,школа І-ІІІ ступенів №175 ім.В Марченка Шевчен...,8039100000,"[2020, 2021, 2022]",80,школа3ступ175менвмарченкашевченквськогорайонумкив,26125940


In [63]:
schools_ed_fin[schools_ed_fin['EDRPOU'] != ''].year.value_counts()

year
[2022]                                        2550
[2021, 2022]                                  1158
[2016, 2017, 2018, 2019, 2020]                 983
[2016, 2017, 2018, 2019, 2020, 2021, 2022]     981
[2016, 2017, 2018, 2019, 2020, 2021]           806
                                              ... 
[2016, 2020, 2021, 2022]                         1
[2016, 2017, 2019, 2020, 2022]                   1
[2018, 2022]                                     1
[2016, 2018, 2019, 2020, 2022]                   1
[2017, 2021, 2022]                               1
Name: count, Length: 107, dtype: int64

# Finance 2020

In [64]:
schools_ed_fin_new = schools_ed_fin
schools_ed_fin_new

,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU
0,Славгородська загальноосвітня школа I-II...,5922386401,"[2016, 2017]",59,славгородськазагальноосвтняшкола3ступкраснопль...,32758871
1,"Івашківський навчально-виховний комплекс ""Заг...",5122582601,[2016],51,вашквськийнавчальновиховнийкомплексзагальноосв...,
2,"Грабівський навчально-виховний комплекс ""Зага...",5122581601,[2016],51,грабвськийнавчальновиховнийкомплексзагальноосв...,
3,"Кримківська ордена ""Знак Пошани"" ЗОШ І-ІІІ ст...",4825482206,[2016],48,кримквськаорденазнакпошанизагальноосвтняшкола3...,
4,Лип'янська загальноосвітня школа І-ІІІ ступен...,7125784001,[2016],71,липянськазагальноосвтняшкола3ступшполянськорай...,
...,...,...,...,...,...,...
27505,філія комунального опорного закладу освіти «Чо...,5123184201,[2019],51,флякомунальногоопорногозакладуосвтичорнянськаз...,
27506,"філія навчально-виховного комплексу ""Голованів...",3521482801,[2017],35,флянавчальновиховногокомплексуголованвськазага...,
27507,школа І-ІІІ ступенів № 104 імені О.Ольжича Обо...,8038000000,"[2018, 2019, 2020, 2021, 2022]",80,школа3ступ104меноольжичаоболонськогорайонумкив,22880852
27508,школа І-ІІІ ступенів №175 ім.В Марченка Шевчен...,8039100000,"[2020, 2021, 2022]",80,школа3ступ175менвмарченкашевченквськогорайонумкив,26125940


In [65]:
finance_data_2020 =  pd.read_excel('/Users/scipyguru/Library/Mobile Documents/com~apple~CloudDocs/Documents_New/NY fairness/NeurIPS/ZNO-Dataset/notebooks/school_level/fin_data/ZZSO-2020.xlsx', header = 1, dtype='str')
finance_data_2020

,№ Області,Область,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (касові видатки)","Видатки на функціонування закладу (загальний фонд), тис.грн.\n2021 рік (план)","Видатки на 1 учня, 2020, тис. грн.","Видатки на 1 клас, тис. грн. 2020 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,ЄДРПОУ,Примітка
0,2,Вінницька,місто,1,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,...,4388.111,5006.502,37.50522222222222,487.5678888888889,NaN,02553000000,комунальна,7137,25844555,NaN
1,2,Вінницька,місто,2,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,...,6523.777,8614.601,20.909541666666666,543.6480833333334,NaN,02553000000,комунальна,6936,25844992,NaN
2,2,Вінницька,місто,3,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,...,9273.777,12627.197,17.49769245283019,441.6084285714286,NaN,02553000000,комунальна,7149,25844928,NaN
3,2,Вінницька,місто,4,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,...,8654.501,11029.117,18.61183010752688,455.50005263157897,NaN,02553000000,комунальна,7191,25844970,NaN
4,2,Вінницька,місто,5,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,...,14833.298,19559.764,18.08938780487805,478.49348387096774,NaN,02553000000,комунальна,6976,23101028,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13453,26,місто Київ,місто,13454,місто Київ,"Спеціалізований навчально-виховний комплекс ""К...",I,NaN,11,21,...,5415.94732,6742.1,32.626188674698795,902.6578866666667,NaN,26000000000,комунальна,22986,22874627,NaN
13454,26,місто Київ,місто,13455,місто Київ,"НВК ""Спеціалізована школа І ступеня з поглибле...",I,NaN,14,21,...,9401.191,11418.88,44.76757619047619,1343.0272857142859,NaN,26000000000,комунальна,23112,21580289,NaN
13455,26,місто Київ,місто,13456,місто Київ,Початкова школа № 334 Дарницького району м. Києва,I,NaN,24,34,...,9552.78,15143.279,21.275679287305124,682.3414285714287,NaN,26000000000,комунальна,23731,42384408,NaN
13456,26,місто Київ,місто,13457,місто Київ,Школа І ступеня № 333 Дарницького району м. Києва,I-III,NaN,29,23,...,11503.71,15556.951,19.664461538461538,718.981875,NaN,26000000000,комунальна,23680,41659088,NaN


In [66]:
create_id(finance_data_2020, 'Найменування закладу', 'id')

,№ Області,Область,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на функціонування закладу (загальний фонд), тис.грн.\n2021 рік (план)","Видатки на 1 учня, 2020, тис. грн.","Видатки на 1 клас, тис. грн. 2020 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,ЄДРПОУ,Примітка,id
0,2,Вінницька,місто,1,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,...,5006.502,37.50522222222222,487.5678888888889,NaN,02553000000,комунальна,7137,25844555,NaN,загальноосвтнйнавчальновиховнийкомплекс3ступшк...
1,2,Вінницька,місто,2,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,...,8614.601,20.909541666666666,543.6480833333334,NaN,02553000000,комунальна,6936,25844992,NaN,кнззагальноосвтняшкола3ступ3козятинськомськора...
2,2,Вінницька,місто,3,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,...,12627.197,17.49769245283019,441.6084285714286,NaN,02553000000,комунальна,7149,25844928,NaN,комунальнийнавчальнийзакладзагальноосвтняшкола...
3,2,Вінницька,місто,4,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,...,11029.117,18.61183010752688,455.50005263157897,NaN,02553000000,комунальна,7191,25844970,NaN,загальноосвтнясередняшкола3ступ5козятинськомсь...
4,2,Вінницька,місто,5,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,...,19559.764,18.08938780487805,478.49348387096774,NaN,02553000000,комунальна,6976,23101028,NaN,комунальнийнавчальнийзакладспецалзованашкола3с...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13453,26,місто Київ,місто,13454,місто Київ,"Спеціалізований навчально-виховний комплекс ""К...",I,NaN,11,21,...,6742.1,32.626188674698795,902.6578866666667,NaN,26000000000,комунальна,22986,22874627,NaN,спецалзованийнавчальновиховнийкомплекскивськка...
13454,26,місто Київ,місто,13455,місто Київ,"НВК ""Спеціалізована школа І ступеня з поглибле...",I,NaN,14,21,...,11418.88,44.76757619047619,1343.0272857142859,NaN,26000000000,комунальна,23112,21580289,NaN,навчальновиховнийкомплексспецалзованашколаступ...
13455,26,місто Київ,місто,13456,місто Київ,Початкова школа № 334 Дарницького району м. Києва,I,NaN,24,34,...,15143.279,21.275679287305124,682.3414285714287,NaN,26000000000,комунальна,23731,42384408,NaN,початковашкола334дарницькогорайонумкив
13456,26,місто Київ,місто,13457,місто Київ,Школа І ступеня № 333 Дарницького району м. Києва,I-III,NaN,29,23,...,15556.951,19.664461538461538,718.981875,NaN,26000000000,комунальна,23680,41659088,NaN,школаступ333дарницькогорайонумкив


In [67]:
finance_data_2020

,№ Області,Область,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на функціонування закладу (загальний фонд), тис.грн.\n2021 рік (план)","Видатки на 1 учня, 2020, тис. грн.","Видатки на 1 клас, тис. грн. 2020 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,ЄДРПОУ,Примітка,id
0,2,Вінницька,місто,1,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,...,5006.502,37.50522222222222,487.5678888888889,NaN,02553000000,комунальна,7137,25844555,NaN,загальноосвтнйнавчальновиховнийкомплекс3ступшк...
1,2,Вінницька,місто,2,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,...,8614.601,20.909541666666666,543.6480833333334,NaN,02553000000,комунальна,6936,25844992,NaN,кнззагальноосвтняшкола3ступ3козятинськомськора...
2,2,Вінницька,місто,3,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,...,12627.197,17.49769245283019,441.6084285714286,NaN,02553000000,комунальна,7149,25844928,NaN,комунальнийнавчальнийзакладзагальноосвтняшкола...
3,2,Вінницька,місто,4,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,...,11029.117,18.61183010752688,455.50005263157897,NaN,02553000000,комунальна,7191,25844970,NaN,загальноосвтнясередняшкола3ступ5козятинськомсь...
4,2,Вінницька,місто,5,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,...,19559.764,18.08938780487805,478.49348387096774,NaN,02553000000,комунальна,6976,23101028,NaN,комунальнийнавчальнийзакладспецалзованашкола3с...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13453,26,місто Київ,місто,13454,місто Київ,"Спеціалізований навчально-виховний комплекс ""К...",I,NaN,11,21,...,6742.1,32.626188674698795,902.6578866666667,NaN,26000000000,комунальна,22986,22874627,NaN,спецалзованийнавчальновиховнийкомплекскивськка...
13454,26,місто Київ,місто,13455,місто Київ,"НВК ""Спеціалізована школа І ступеня з поглибле...",I,NaN,14,21,...,11418.88,44.76757619047619,1343.0272857142859,NaN,26000000000,комунальна,23112,21580289,NaN,навчальновиховнийкомплексспецалзованашколаступ...
13455,26,місто Київ,місто,13456,місто Київ,Початкова школа № 334 Дарницького району м. Києва,I,NaN,24,34,...,15143.279,21.275679287305124,682.3414285714287,NaN,26000000000,комунальна,23731,42384408,NaN,початковашкола334дарницькогорайонумкив
13456,26,місто Київ,місто,13457,місто Київ,Школа І ступеня № 333 Дарницького району м. Києва,I-III,NaN,29,23,...,15556.951,19.664461538461538,718.981875,NaN,26000000000,комунальна,23680,41659088,NaN,школаступ333дарницькогорайонумкив


In [68]:
finance_data_2020.columns

Index(['№ Області', 'Область', 'Тип школи: міська / сільська', '№№',
       'Назва тер. Одиниці', 'Найменування закладу',
       'Ступінь закладу ( I, I-II, I-III)',
       'Загальна площа приміщень школи ( без господарських будівель) (кв.м.)',
       'Кількість штатних працівників педагогічного персоналу  разом',
       'Кількість штатних працівників непедагогічного персоналу  разом',
       'Кількість штатних працівників педагогічного персоналу  пенсійного віку',
       'Питома вага непедагогічного персоналу у загальній кількості працівників, %',
       'Питома вага педагогічних працівників пенсійного віку у загальній кількості педагогічних працівників, %',
       'Кількість учнів разом', 'Кількість класів разом',
       'Фактична наповнюваність класів, учнів на клас',
       'Показник співвідношення кількості учнів на одного вчителя, учнів на вчителя',
       'Видатки на функціонування закладу (загальний фонд), тис.грн.\n2020 рік (касові видатки)',
       'Видатки на функціонування 

In [69]:
finance_data_2020.rename(columns={'ЄДРПОУ':'EDRPOU'}, inplace=True)

In [70]:
finance_data_2020

,№ Області,Область,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на функціонування закладу (загальний фонд), тис.грн.\n2021 рік (план)","Видатки на 1 учня, 2020, тис. грн.","Видатки на 1 клас, тис. грн. 2020 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,EDRPOU,Примітка,id
0,2,Вінницька,місто,1,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,...,5006.502,37.50522222222222,487.5678888888889,NaN,02553000000,комунальна,7137,25844555,NaN,загальноосвтнйнавчальновиховнийкомплекс3ступшк...
1,2,Вінницька,місто,2,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,...,8614.601,20.909541666666666,543.6480833333334,NaN,02553000000,комунальна,6936,25844992,NaN,кнззагальноосвтняшкола3ступ3козятинськомськора...
2,2,Вінницька,місто,3,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,...,12627.197,17.49769245283019,441.6084285714286,NaN,02553000000,комунальна,7149,25844928,NaN,комунальнийнавчальнийзакладзагальноосвтняшкола...
3,2,Вінницька,місто,4,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,...,11029.117,18.61183010752688,455.50005263157897,NaN,02553000000,комунальна,7191,25844970,NaN,загальноосвтнясередняшкола3ступ5козятинськомсь...
4,2,Вінницька,місто,5,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,...,19559.764,18.08938780487805,478.49348387096774,NaN,02553000000,комунальна,6976,23101028,NaN,комунальнийнавчальнийзакладспецалзованашкола3с...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13453,26,місто Київ,місто,13454,місто Київ,"Спеціалізований навчально-виховний комплекс ""К...",I,NaN,11,21,...,6742.1,32.626188674698795,902.6578866666667,NaN,26000000000,комунальна,22986,22874627,NaN,спецалзованийнавчальновиховнийкомплекскивськка...
13454,26,місто Київ,місто,13455,місто Київ,"НВК ""Спеціалізована школа І ступеня з поглибле...",I,NaN,14,21,...,11418.88,44.76757619047619,1343.0272857142859,NaN,26000000000,комунальна,23112,21580289,NaN,навчальновиховнийкомплексспецалзованашколаступ...
13455,26,місто Київ,місто,13456,місто Київ,Початкова школа № 334 Дарницького району м. Києва,I,NaN,24,34,...,15143.279,21.275679287305124,682.3414285714287,NaN,26000000000,комунальна,23731,42384408,NaN,початковашкола334дарницькогорайонумкив
13456,26,місто Київ,місто,13457,місто Київ,Школа І ступеня № 333 Дарницького району м. Києва,I-III,NaN,29,23,...,15556.951,19.664461538461538,718.981875,NaN,26000000000,комунальна,23680,41659088,NaN,школаступ333дарницькогорайонумкив


In [71]:
finance_data_2020[(finance_data_2020.EDRPOU.str.len() < 8)&(finance_data_2020.EDRPOU.str.len().notna())]

,№ Області,Область,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на функціонування закладу (загальний фонд), тис.грн.\n2021 рік (план)","Видатки на 1 учня, 2020, тис. грн.","Видатки на 1 клас, тис. грн. 2020 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,EDRPOU,Примітка,id
17,2,Вінницька,село,18,Копайгородська селищна ТГ,Каришківська загальноосвітня школа І-ІІІ ступенів,I-III,NaN,20,16,...,5847.5,51.04597701149425,403.72727272727275,NaN,02554000000,комунальна,6927,2141288,NaN,каришквськазагальноосвтняшкола3ступен
27,2,Вінницька,село,28,Барcька міська ТГ,Войнашівська загальноосвітня школа І-ІІІ ступе...,I-III,NaN,45,25,...,13843,28.060243902439026,426.1,опорна,02505000000,комунальна,6875,2141288,NaN,войнашвськазагальноосвтняшкола3ступен
37,2,Вінницька,село,38,Копайгородська селищна ТГ,Лісівська загальноосвітня школа І ступеня,I,NaN,5,4,...,1533,44.95652173913044,344.6666666666667,NaN,02554000000,комунальна,6918,2141288,NaN,лсвськазагальноосвтняшколаступен
47,2,Вінницька,місто,48,Бершадська міська ТГ,Бершадська загальноосвітня школа І-ІІІ ступені...,I-III,NaN,54,30,...,17284.594,22.93234375,497.95946428571426,NaN,02548000000,комунальна,6152,2141294,NaN,бершадськазагальноосвтняшкола3ступ
60,2,Вінницька,село,61,Бершадська міська ТГ,Флоринська загальноосвітня школа І -ІІІ ст.,I-III,NaN,30,19,...,9907.135,22.338088471849865,416.60535,NaN,02548000000,комунальна,6144,2141294,NaN,флоринськазагальноосвтняшкола3ст
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13352,26,місто Київ,місто,13353,місто Київ,Спеціалізована школа І-ІІІ ступенів №152 з пог...,I-III,NaN,44,30,...,22593.8,32.26361112244898,790.4584725,NaN,26000000000,комунальна,22933,3875118,NaN,спецалзованашкола3ступ152зпоглибленимвивченяма...
13360,26,місто Київ,місто,13361,місто Київ,Школа І-ІІІ ступенів № 294 Десянського району ...,I-III,NaN,62,33,...,31025.2,24.475661793286218,675.7670524390243,NaN,26000000000,комунальна,22960,1489305,NaN,школа3ступ294десянськогорайонумкив
13394,26,місто Київ,місто,13395,місто Київ,Школа І-ІІІ ступенів № 282 Деснянського району...,I-III,NaN,55,29,...,29592.3,24.102524337101748,710.6592781818182,NaN,26000000000,комунальна,22956,1488067,NaN,школа3ступ282деснянськогорайонумкив
13430,26,місто Київ,місто,13431,місто Київ,Спеціалізована школа І-ІІІ ступенів №202 з пог...,I-III,NaN,48,29,...,22414,25.32127277322405,639.1438506896553,NaN,26000000000,комунальна,22937,3875354,NaN,спецалзованашкола3ступ202зпоглибленимвивченямп...


In [72]:
finance_data_2020['EDRPOU']=finance_data_2020['EDRPOU'].apply(lambda x: '0'+x if (isinstance(x, str) and 0<len(x)<8) else x)

In [73]:
finance_data_2020['EDRPOU']=finance_data_2020['EDRPOU'].apply(lambda x: '0'+x if (isinstance(x, str) and 0<len(x)<8) else x)

In [74]:
finance_data_2020['EDRPOU']=finance_data_2020['EDRPOU'].apply(lambda x: '0'+x if (isinstance(x, str) and 0<len(x)<8) else x)

In [75]:
finance_data_2020[finance_data_2020.EDRPOU.str.len() != 8].EDRPOU

191     0007995
198        0000
521       00011
3371       0000
5666       0000
6495       0000
6498       0000
6499       0000
6716       0000
Name: EDRPOU, dtype: object

In [76]:
finance_data_2020 = finance_data_2020[finance_data_2020.EDRPOU.str.len() == 8]

In [77]:
set(finance_data_2020['Область'].unique())-set(locations.regname.unique())

{'Івано-Франківська',
 'Волинська',
 'Вінницька',
 'Дніпропетровська',
 'Донецька',
 'Житомирська',
 'Закарпатська',
 'Запорізька',
 'Київська',
 'Кіровоградська',
 'Луганська',
 'Львівська',
 'Миколаївська',
 'Одеська',
 'Полтавська',
 'Рівненська',
 'Сумська',
 'Тернопільська',
 'Харківська',
 'Херсонська',
 'Хмельницька',
 'Черкаська',
 'Чернівецька',
 'Чернігівська',
 'місто Київ'}

In [78]:
finance_data_2020['Область'] = finance_data_2020['Область'].apply(lambda x: 'м.Київ' if x == 'місто Київ' else x+' область')
finance_data_2020

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/3892411677.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finance_data_2020['Область'] = finance_data_2020['Область'].apply(lambda x: 'м.Київ' if x == 'місто Київ' else x+' область')


,№ Області,Область,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на функціонування закладу (загальний фонд), тис.грн.\n2021 рік (план)","Видатки на 1 учня, 2020, тис. грн.","Видатки на 1 клас, тис. грн. 2020 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,EDRPOU,Примітка,id
0,2,Вінницька область,місто,1,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,...,5006.502,37.50522222222222,487.5678888888889,NaN,02553000000,комунальна,7137,25844555,NaN,загальноосвтнйнавчальновиховнийкомплекс3ступшк...
1,2,Вінницька область,місто,2,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,...,8614.601,20.909541666666666,543.6480833333334,NaN,02553000000,комунальна,6936,25844992,NaN,кнззагальноосвтняшкола3ступ3козятинськомськора...
2,2,Вінницька область,місто,3,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,...,12627.197,17.49769245283019,441.6084285714286,NaN,02553000000,комунальна,7149,25844928,NaN,комунальнийнавчальнийзакладзагальноосвтняшкола...
3,2,Вінницька область,місто,4,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,...,11029.117,18.61183010752688,455.50005263157897,NaN,02553000000,комунальна,7191,25844970,NaN,загальноосвтнясередняшкола3ступ5козятинськомсь...
4,2,Вінницька область,місто,5,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,...,19559.764,18.08938780487805,478.49348387096774,NaN,02553000000,комунальна,6976,23101028,NaN,комунальнийнавчальнийзакладспецалзованашкола3с...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13453,26,м.Київ,місто,13454,місто Київ,"Спеціалізований навчально-виховний комплекс ""К...",I,NaN,11,21,...,6742.1,32.626188674698795,902.6578866666667,NaN,26000000000,комунальна,22986,22874627,NaN,спецалзованийнавчальновиховнийкомплекскивськка...
13454,26,м.Київ,місто,13455,місто Київ,"НВК ""Спеціалізована школа І ступеня з поглибле...",I,NaN,14,21,...,11418.88,44.76757619047619,1343.0272857142859,NaN,26000000000,комунальна,23112,21580289,NaN,навчальновиховнийкомплексспецалзованашколаступ...
13455,26,м.Київ,місто,13456,місто Київ,Початкова школа № 334 Дарницького району м. Києва,I,NaN,24,34,...,15143.279,21.275679287305124,682.3414285714287,NaN,26000000000,комунальна,23731,42384408,NaN,початковашкола334дарницькогорайонумкив
13456,26,м.Київ,місто,13457,місто Київ,Школа І ступеня № 333 Дарницького району м. Києва,I-III,NaN,29,23,...,15556.951,19.664461538461538,718.981875,NaN,26000000000,комунальна,23680,41659088,NaN,школаступ333дарницькогорайонумкив


In [79]:
finance_data_2020.rename(columns={'Область':'regname'}, inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/2355463563.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finance_data_2020.rename(columns={'Область':'regname'}, inplace=True)


In [80]:
finance_data_2020

,№ Області,regname,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на функціонування закладу (загальний фонд), тис.грн.\n2021 рік (план)","Видатки на 1 учня, 2020, тис. грн.","Видатки на 1 клас, тис. грн. 2020 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,EDRPOU,Примітка,id
0,2,Вінницька область,місто,1,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,...,5006.502,37.50522222222222,487.5678888888889,NaN,02553000000,комунальна,7137,25844555,NaN,загальноосвтнйнавчальновиховнийкомплекс3ступшк...
1,2,Вінницька область,місто,2,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,...,8614.601,20.909541666666666,543.6480833333334,NaN,02553000000,комунальна,6936,25844992,NaN,кнззагальноосвтняшкола3ступ3козятинськомськора...
2,2,Вінницька область,місто,3,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,...,12627.197,17.49769245283019,441.6084285714286,NaN,02553000000,комунальна,7149,25844928,NaN,комунальнийнавчальнийзакладзагальноосвтняшкола...
3,2,Вінницька область,місто,4,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,...,11029.117,18.61183010752688,455.50005263157897,NaN,02553000000,комунальна,7191,25844970,NaN,загальноосвтнясередняшкола3ступ5козятинськомсь...
4,2,Вінницька область,місто,5,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,...,19559.764,18.08938780487805,478.49348387096774,NaN,02553000000,комунальна,6976,23101028,NaN,комунальнийнавчальнийзакладспецалзованашкола3с...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13453,26,м.Київ,місто,13454,місто Київ,"Спеціалізований навчально-виховний комплекс ""К...",I,NaN,11,21,...,6742.1,32.626188674698795,902.6578866666667,NaN,26000000000,комунальна,22986,22874627,NaN,спецалзованийнавчальновиховнийкомплекскивськка...
13454,26,м.Київ,місто,13455,місто Київ,"НВК ""Спеціалізована школа І ступеня з поглибле...",I,NaN,14,21,...,11418.88,44.76757619047619,1343.0272857142859,NaN,26000000000,комунальна,23112,21580289,NaN,навчальновиховнийкомплексспецалзованашколаступ...
13455,26,м.Київ,місто,13456,місто Київ,Початкова школа № 334 Дарницького району м. Києва,I,NaN,24,34,...,15143.279,21.275679287305124,682.3414285714287,NaN,26000000000,комунальна,23731,42384408,NaN,початковашкола334дарницькогорайонумкив
13456,26,м.Київ,місто,13457,місто Київ,Школа І ступеня № 333 Дарницького району м. Києва,I-III,NaN,29,23,...,15556.951,19.664461538461538,718.981875,NaN,26000000000,комунальна,23680,41659088,NaN,школаступ333дарницькогорайонумкив


In [81]:
finance_data_2020 = finance_data_2020.merge(loc[['regname', 'KOATUU_2020_reg']], on=['regname'], how='left')
finance_data_2020.head()

,№ Області,regname,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на 1 учня, 2020, тис. грн.","Видатки на 1 клас, тис. грн. 2020 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,EDRPOU,Примітка,id,KOATUU_2020_reg
0,2,Вінницька область,місто,1,Козятинська міська ТГ,Загальноосвітній навчально-виховний комплекс І...,I-III,NaN,16,19,...,37.50522222222222,487.5678888888889,NaN,02553000000,комунальна,7137,25844555,NaN,загальноосвтнйнавчальновиховнийкомплекс3ступшк...,05
1,2,Вінницька область,місто,2,Козятинська міська ТГ,"КНЗ ""Загальноосвітня школа І-ІІІ ступенів №3 К...",I-III,NaN,27,15,...,20.909541666666666,543.6480833333334,NaN,02553000000,комунальна,6936,25844992,NaN,кнззагальноосвтняшкола3ступ3козятинськомськора...,05
2,2,Вінницька область,місто,3,Козятинська міська ТГ,"Комунальний навчальний заклад ""Загальноосвітня...",I-III,NaN,39,17,...,17.49769245283019,441.6084285714286,NaN,02553000000,комунальна,7149,25844928,NaN,комунальнийнавчальнийзакладзагальноосвтняшкола...,05
3,2,Вінницька область,місто,4,Козятинська міська ТГ,Загальноосвітня середня школа І-ІІІ ступенів №...,I-III,NaN,35,18,...,18.61183010752688,455.50005263157897,NaN,02553000000,комунальна,7191,25844970,NaN,загальноосвтнясередняшкола3ступ5козятинськомсь...,05
4,2,Вінницька область,місто,5,Козятинська міська ТГ,"Комунальний навчальний заклад ""Спеціалізована ...",I-III,NaN,55,21,...,18.08938780487805,478.49348387096774,NaN,02553000000,комунальна,6976,23101028,NaN,комунальнийнавчальнийзакладспецалзованашкола3с...,05


In [82]:
finance_data_2020[finance_data_2020.KOATUU_2020_reg.isna()]

,№ Області,regname,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на 1 учня, 2020, тис. грн.","Видатки на 1 клас, тис. грн. 2020 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,EDRPOU,Примітка,id,KOATUU_2020_reg


In [83]:
finance_data_2020.shape

(13449, 29)

In [84]:
finance_data_2020[finance_data_2020.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)]

,№ Області,regname,Тип школи: міська / сільська,№№,Назва тер. Одиниці,Найменування закладу,"Ступінь закладу ( I, I-II, I-III)",Загальна площа приміщень школи ( без господарських будівель) (кв.м.),Кількість штатних працівників педагогічного персоналу разом,Кількість штатних працівників непедагогічного персоналу разом,...,"Видатки на 1 учня, 2020, тис. грн.","Видатки на 1 клас, тис. грн. 2020 рік (каса)",Опорна,Код бюджету,Форма власності,Код ДІСО,EDRPOU,Примітка,id,KOATUU_2020_reg
43,2,Вінницька область,місто,44,Бершадська міська ТГ,Бершадська загальноосвітня школа І-ІІІ ступені...,I-III,NaN,51,27,...,20.47727617801047,504.6657741935484,NaN,02548000000,комунальна,6124,23056479,NaN,бершадськазагальноосвтняшкола3ступ,05
47,2,Вінницька область,місто,48,Бершадська міська ТГ,Бершадська загальноосвітня школа І-ІІІ ступені...,I-III,NaN,54,30,...,22.93234375,497.95946428571426,NaN,02548000000,комунальна,6152,02141294,NaN,бершадськазагальноосвтняшкола3ступ,05
52,2,Вінницька область,місто,53,Бершадська міська ТГ,Бершадська загальноосвітня школа І-ІІІ ступені...,I-III,NaN,30,14,...,29.59259243697479,370.6861578947369,NaN,02548000000,комунальна,6110,23056463,NaN,бершадськазагальноосвтняшкола3ступ,05
543,2,Вінницька область,місто,547,Літинська селищна ТГ,Літинська середня загальноосвітня школаІ-ІІІст...,I-III,NaN,45,14,...,18.731711409395974,429.38846153846157,NaN,02541000000,комунальна,8003,02141383,NaN,лтинськасереднязагальноосвтняшкола3ступ,05
544,2,Вінницька область,місто,548,Літинська селищна ТГ,Літинська середня загальноосвітня школа І-ІІІ ...,I-III,NaN,45,24,...,14.575399361022365,350.9307692307693,NaN,02541000000,комунальна,7981,25497740,NaN,лтинськасереднязагальноосвтняшкола3ступ,05
1522,4,Дніпропетровська область,місто,1526,Першотравенська міська ТГ,Першотравенська загальноосвітня школа І-ІІІ ст...,I-III,NaN,49,22,...,16.97035066505441,438.5775,NaN,04585000000,комунальна,2751,26329200,NaN,першотравенськазагальноосвтняшкола3ступ,12
1523,4,Дніпропетровська область,місто,1527,Першотравенська міська ТГ,Першотравенська загальноосвітня школа І-ІІІ ст...,I-III,NaN,28,17,...,18.6965,427.3485714285714,NaN,04585000000,комунальна,2727,19433836,NaN,першотравенськазагальноосвтняшкола3ступ,12
1524,4,Дніпропетровська область,місто,1528,Першотравенська міська ТГ,Першотравенська загальноосвітня школа І-ІІІ ст...,I-III,NaN,30,16,...,17.095703564727955,414.18227272727273,NaN,04585000000,комунальна,3272,19433842,NaN,першотравенськазагальноосвтняшкола3ступ,12
1525,4,Дніпропетровська область,місто,1529,Першотравенська міська ТГ,Першотравенська загальноосвітня школа І-ІІІ ст...,I-III,NaN,44,19,...,18.076782246879336,449.4262068965517,NaN,04585000000,комунальна,2732,19433824,NaN,першотравенськазагальноосвтняшкола3ступ,12
1526,4,Дніпропетровська область,місто,1530,Першотравенська міська ТГ,Першотравенська загальноосвітня школа І-ІІІ ст...,I-III,NaN,31,14,...,19.852601809954752,438.7425,NaN,04585000000,комунальна,2722,26329192,NaN,першотравенськазагальноосвтняшкола3ступ,12


In [85]:
finance_data_2020.drop(finance_data_2020[finance_data_2020.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)].index.values.tolist()  , inplace=True)

In [86]:
schools_ed_fin_new[schools_ed_fin_new['EDRPOU'] != '']

,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU
0,Славгородська загальноосвітня школа I-II...,5922386401,"[2016, 2017]",59,славгородськазагальноосвтняшкола3ступкраснопль...,32758871
7,"""Баламутівський заклад загальної середньої осв...",6825880801,[2021],68,баламутвськийзакладзагальносередньоосвти3ступх...,22764494
10,"""Великолазівський ліцей"" Баранинської сільсько...",2124880305,"[2021, 2022]",21,великолазвськийлцейбаранинськосльськорадизакар...,43758931
13,"""Глинівецький ліцей"" Червоненської селищної ра...",1820355602,[2022],18,глинвецькийлцейчервоненськоселищнорадижитомирс...,20408447
14,"""Демино-Олександрівський ліцей Троїцької селищ...",4425455109,"[2019, 2020, 2021]",44,деминоолександрвськийлцейтроцькоселищнорадитро...,20184835
...,...,...,...,...,...,...
27493,"опорний навчальний заклад ""Левківська загально...",1822084101,"[2020, 2021]",18,опорнийнавчальнийзакладлевквськазагальноосвтня...,22060758
27496,середня загальноосвітня школа №67 м. Львова,4610136300,[2021],46,середнязагальноосвтняшкола67мльвов,22350249
27507,школа І-ІІІ ступенів № 104 імені О.Ольжича Обо...,8038000000,"[2018, 2019, 2020, 2021, 2022]",80,школа3ступ104меноольжичаоболонськогорайонумкив,22880852
27508,школа І-ІІІ ступенів №175 ім.В Марченка Шевчен...,8039100000,"[2020, 2021, 2022]",80,школа3ступ175менвмарченкашевченквськогорайонумкив,26125940


In [87]:
schools_ed_fin_new2 = merging(schools_ed_fin_new, finance_data_2020[['KOATUU_2020_reg', 'id', 'EDRPOU']], ['KOATUU_2020_reg', 'id'])
schools_ed_fin_new2

Percentage matched: 48.6 %


,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU
0,Славгородська загальноосвітня школа I-II...,5922386401,"[2016, 2017]",59,славгородськазагальноосвтняшкола3ступкраснопль...,32758871
1,"Івашківський навчально-виховний комплекс ""Заг...",5122582601,[2016],51,вашквськийнавчальновиховнийкомплексзагальноосв...,
2,"Грабівський навчально-виховний комплекс ""Зага...",5122581601,[2016],51,грабвськийнавчальновиховнийкомплексзагальноосв...,
3,"Кримківська ордена ""Знак Пошани"" ЗОШ І-ІІІ ст...",4825482206,[2016],48,кримквськаорденазнакпошанизагальноосвтняшкола3...,
4,Лип'янська загальноосвітня школа І-ІІІ ступен...,7125784001,[2016],71,липянськазагальноосвтняшкола3ступшполянськорай...,24355707
...,...,...,...,...,...,...
27505,філія комунального опорного закладу освіти «Чо...,5123184201,[2019],51,флякомунальногоопорногозакладуосвтичорнянськаз...,
27506,"філія навчально-виховного комплексу ""Голованів...",3521482801,[2017],35,флянавчальновиховногокомплексуголованвськазага...,
27507,школа І-ІІІ ступенів № 104 імені О.Ольжича Обо...,8038000000,"[2018, 2019, 2020, 2021, 2022]",80,школа3ступ104меноольжичаоболонськогорайонумкив,22880852
27508,школа І-ІІІ ступенів №175 ім.В Марченка Шевчен...,8039100000,"[2020, 2021, 2022]",80,школа3ступ175менвмарченкашевченквськогорайонумкив,26125940


In [88]:
schools_ed_fin_new2[schools_ed_fin_new2['EDRPOU'] == ''].year.value_counts()

year
[2016]                            2563
[2022]                            1438
[2016, 2017]                      1362
[2016, 2017, 2018]                 925
[2021, 2022]                       883
                                  ... 
[2016, 2019, 2020, 2021, 2022]       1
[2017, 2020, 2021, 2022]             1
[2016, 2017, 2018, 2021, 2022]       1
[2018, 2022]                         1
[2016, 2018, 2020, 2022]             1
Name: count, Length: 88, dtype: int64

# Universities

In [89]:
univ_data = pd.read_excel('./school_data/zakladi-vishchoyi-osviti-09.xlsx', dtype = str)
univ_data

,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,Ідентифікаційний код,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Поштовий індекс (юридична),Код КОАТУУ (юридична),Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника"
0,"Академія праці, соціальних відносин і туризму",53,NaN,АПСВТ,"Academy of Labour, Social Relations and Tourism",1993,04641405,Заклад вищої освіти,Академія,Приватна,...,03680,8000000000,КИЇВ,Київ,"вул. Велика Окружна дорога, 3",+38(044) 522-49-40,info@socosvita.kiev.ua,http://www. socosvita.kiev.ua,Ректор,Буяшенко Вікторія Василівна
1,Бізнес-коледж Приватного вищого навчального за...,3555,228,БК ПВНЗ «Європейський університет»,Business College of Private Higher Educational...,2002,40487422,Заклад вищої освіти,Коледж,Приватна,...,03115,8000000000,КИЇВ,Київ,"бульв. Академіка Вернадського, 16-В",+380(44)-432-00-58,bkeu@kyiv.e-u.in.ua,www.e-u.in.ua,директор,Сильницький Юрій Вікторович
2,"Вищий навчальний заклад ""Відкритий міжнародний...",217,NaN,"Університет ""Україна""",Higher Education Institution 'Open Internation...,1998,30373644,Заклад вищої освіти,Університет,Приватна,...,04071,8038500000,КИЇВ,Київ,"вул. Хорива, 1Г",(044) 4287410; 4252322; 4225019; 4252926,viddilorgrobotu@ukr.net,www.vmurol.com.ua,президент,Таланчук Петро Михайлович
3,"Вищий навчальний заклад Інститут дизайну, архі...",1319,NaN,"Вищий навчальний заклад ""Інститут дизайну, арх...",Higher educational institution “Institute of D...,2000,32594570,Заклад вищої освіти,Інститут,Приватна,...,03039,8038500000,КИЇВ,Київ,"вул. Фрометівська, 2",+380(44)-496-97-60,fb.mail.ir@gmail.com,www.ir.edu.ua,В. о. директора,Бойко Світлана Станіславівна
4,"Вищий навчальний заклад ""Київська Академія пер...",865,NaN,"ВНЗ ""КАПМ""","Higher educational institution ""Kyiv Academy o...",2002,26199252,Заклад вищої освіти,Академія,Приватна,...,03067,8038900000,КИЇВ,Київ,"вул. Виборзька, 86",0444555672,kapm@ukr.net,www.kapi.com.ua,ректор,Лубянська Світлана Павлівна
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,Відокремлений підрозділ національного універси...,2839,7,.,NaN,2000,35139007,Заклад вищої освіти,Відокремлений підрозділ,Державна,...,17400,7420610100,Чернігівська область,Бобровиця,"вул. Чернігівська, 19",NaN,NaN,NaN,NaN,NaN
1416,Борзнянський державний сільськогосподарський т...,390,NaN,БДСГТ,State agricultural technical school of Borzna,1999,00729178,Заклад вищої освіти,Технікум (училище),Державна,...,16400,7420810100,Чернігівська область,Борзна,"Олекси Десняка, 23",21106,koba_22@ukr.net,bdsgt.at.ua,Виконуючий обов`язки директора,Линник Віталій Володимирович
1417,Козелецький коледж ветеринарної медицини Білоц...,990,362,ККВМ БНАУ,"Kozelets College of Veterinary Medicine, Bila ...",1925,00729209,Заклад вищої освіти,Коледж,Державна,...,17000,7422055100,Чернігівська область,Козелець,"вул. Сім'ї Розумовських, 41","(046)462-12-00, (046)464-15-96, (046)462-14-83",techvet2@gmail.com,http://www.techvet.com.ua/,Директор коледжу,Редька Алла Іванівна
1418,Остерський коледж будівництва та дизайну,925,NaN,ОКБД,Oster college of civil engineering and designing,1945,01355708,Заклад вищої освіти,Коледж,Державна,...,17044,7422010400,Чернігівська область,Остер,"вул. 8 Березня, 7-А",(046) 464-32-85,oster_budtex@cg.ukrtel.net,http://okbd.16mb.com,Директор,Бардаченко Тетяна Миколаївна


In [90]:
univ_data_id = univ_data

In [91]:
create_id(univ_data_id, 'Назва закладу освіти', 'id')

,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,Ідентифікаційний код,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Код КОАТУУ (юридична),Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника",id
0,"Академія праці, соціальних відносин і туризму",53,NaN,АПСВТ,"Academy of Labour, Social Relations and Tourism",1993,04641405,Заклад вищої освіти,Академія,Приватна,...,8000000000,КИЇВ,Київ,"вул. Велика Окружна дорога, 3",+38(044) 522-49-40,info@socosvita.kiev.ua,http://www. socosvita.kiev.ua,Ректор,Буяшенко Вікторія Василівна,академяпрацсоцальнихвдносинтуризм
1,Бізнес-коледж Приватного вищого навчального за...,3555,228,БК ПВНЗ «Європейський університет»,Business College of Private Higher Educational...,2002,40487422,Заклад вищої освіти,Коледж,Приватна,...,8000000000,КИЇВ,Київ,"бульв. Академіка Вернадського, 16-В",+380(44)-432-00-58,bkeu@kyiv.e-u.in.ua,www.e-u.in.ua,директор,Сильницький Юрій Вікторович,бзнесколеджприватноговищогонавчальногозакладув...
2,"Вищий навчальний заклад ""Відкритий міжнародний...",217,NaN,"Університет ""Україна""",Higher Education Institution 'Open Internation...,1998,30373644,Заклад вищої освіти,Університет,Приватна,...,8038500000,КИЇВ,Київ,"вул. Хорива, 1Г",(044) 4287410; 4252322; 4225019; 4252926,viddilorgrobotu@ukr.net,www.vmurol.com.ua,президент,Таланчук Петро Михайлович,вищийнавчальнийзакладвдкритиймжнароднийунверси...
3,"Вищий навчальний заклад Інститут дизайну, архі...",1319,NaN,"Вищий навчальний заклад ""Інститут дизайну, арх...",Higher educational institution “Institute of D...,2000,32594570,Заклад вищої освіти,Інститут,Приватна,...,8038500000,КИЇВ,Київ,"вул. Фрометівська, 2",+380(44)-496-97-60,fb.mail.ir@gmail.com,www.ir.edu.ua,В. о. директора,Бойко Світлана Станіславівна,вищийнавчальнийзакладнститутдизайнуархтектурит...
4,"Вищий навчальний заклад ""Київська Академія пер...",865,NaN,"ВНЗ ""КАПМ""","Higher educational institution ""Kyiv Academy o...",2002,26199252,Заклад вищої освіти,Академія,Приватна,...,8038900000,КИЇВ,Київ,"вул. Виборзька, 86",0444555672,kapm@ukr.net,www.kapi.com.ua,ректор,Лубянська Світлана Павлівна,вищийнавчальнийзакладкивськаакадемяперукарсько...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,Відокремлений підрозділ національного універси...,2839,7,.,NaN,2000,35139007,Заклад вищої освіти,Відокремлений підрозділ,Державна,...,7420610100,Чернігівська область,Бобровиця,"вул. Чернігівська, 19",NaN,NaN,NaN,NaN,NaN,вдокремленийпдроздлнацональногоунверситетуборе...
1416,Борзнянський державний сільськогосподарський т...,390,NaN,БДСГТ,State agricultural technical school of Borzna,1999,00729178,Заклад вищої освіти,Технікум (училище),Державна,...,7420810100,Чернігівська область,Борзна,"Олекси Десняка, 23",21106,koba_22@ukr.net,bdsgt.at.ua,Виконуючий обов`язки директора,Линник Віталій Володимирович,борзнянськийдержавнийсльськогосподарськийтехнку
1417,Козелецький коледж ветеринарної медицини Білоц...,990,362,ККВМ БНАУ,"Kozelets College of Veterinary Medicine, Bila ...",1925,00729209,Заклад вищої освіти,Коледж,Державна,...,7422055100,Чернігівська область,Козелець,"вул. Сім'ї Розумовських, 41","(046)462-12-00, (046)464-15-96, (046)462-14-83",techvet2@gmail.com,http://www.techvet.com.ua/,Директор коледжу,Редька Алла Іванівна,козелецькийколеджветеринарномедициниблоцерквсь...
1418,Остерський коледж будівництва та дизайну,925,NaN,ОКБД,Oster college of civil engineering and designing,1945,01355708,Заклад вищої освіти,Коледж,Державна,...,7422010400,Чернігівська область,Остер,"вул. 8 Березня, 7-А",(046) 464-32-85,oster_budtex@cg.ukrtel.net,http://okbd.16mb.com,Директор,Бардаченко Тетяна Миколаївна,остерськийколеджбудвництватадизайн


In [92]:
univ_data_id.columns

Index(['Назва закладу освіти', 'Код', 'Код головного закладу', 'Коротка назва',
       'Назва закладу освіти (англ.)', 'Рік заснування',
       'Ідентифікаційний код', 'Вид закладу освіти', 'Тип закладу освіти',
       'Форма власності',
       'Найменування органу, до сфери управління якого належить заклад освіти',
       'Поштовий індекс (фактична)', 'Код КОАТУУ (фактична)',
       'Регіон (фактична)', 'Населений пункт (фактична)', 'Адреса (фактична)',
       'Поштовий індекс (юридична)', 'Код КОАТУУ (юридична)',
       'Регіон (юридична)', 'Населений пункт (юридична)', 'Адреса (юридична)',
       'Телефон / факс', 'Електронна пошта', 'Веб-сайт', 'Посада керівника',
       'Прізвище, ім'я, по батькові керівника', 'id'],
      dtype='object')

In [93]:
univ_data_id.rename(columns={'Код КОАТУУ (юридична)':'KOATUU_2020', 'Ідентифікаційний код':'EDRPOU'}, inplace=True)

In [94]:
univ_data_id[univ_data_id.KOATUU_2020.str.len() != 10]

,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,EDRPOU,Вид закладу освіти,Тип закладу освіти,Форма власності,...,KOATUU_2020,Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника",id
495,Слов'янський енергобудівний технікум,758,NaN,СЕБТ,Sloviansk Power-Building Technical School,1957,00129343,Заклад вищої освіти,Технікум (училище),Державна,...,NaN,NaN,NaN,"вул. Вчительська, 40","(06262)2-45-72, (06262)2-58-82",sebt@ukr.net,sebt.dn.ua,Директор технікуму,Горозій Михайло Іванович,словянськийенергобудвнийтехнку


In [95]:
univ_data_id['KOATUU_2020_reg'] = univ_data_id['KOATUU_2020'].str[:2]
univ_data_id

,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,EDRPOU,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника",id,KOATUU_2020_reg
0,"Академія праці, соціальних відносин і туризму",53,NaN,АПСВТ,"Academy of Labour, Social Relations and Tourism",1993,04641405,Заклад вищої освіти,Академія,Приватна,...,КИЇВ,Київ,"вул. Велика Окружна дорога, 3",+38(044) 522-49-40,info@socosvita.kiev.ua,http://www. socosvita.kiev.ua,Ректор,Буяшенко Вікторія Василівна,академяпрацсоцальнихвдносинтуризм,80
1,Бізнес-коледж Приватного вищого навчального за...,3555,228,БК ПВНЗ «Європейський університет»,Business College of Private Higher Educational...,2002,40487422,Заклад вищої освіти,Коледж,Приватна,...,КИЇВ,Київ,"бульв. Академіка Вернадського, 16-В",+380(44)-432-00-58,bkeu@kyiv.e-u.in.ua,www.e-u.in.ua,директор,Сильницький Юрій Вікторович,бзнесколеджприватноговищогонавчальногозакладув...,80
2,"Вищий навчальний заклад ""Відкритий міжнародний...",217,NaN,"Університет ""Україна""",Higher Education Institution 'Open Internation...,1998,30373644,Заклад вищої освіти,Університет,Приватна,...,КИЇВ,Київ,"вул. Хорива, 1Г",(044) 4287410; 4252322; 4225019; 4252926,viddilorgrobotu@ukr.net,www.vmurol.com.ua,президент,Таланчук Петро Михайлович,вищийнавчальнийзакладвдкритиймжнароднийунверси...,80
3,"Вищий навчальний заклад Інститут дизайну, архі...",1319,NaN,"Вищий навчальний заклад ""Інститут дизайну, арх...",Higher educational institution “Institute of D...,2000,32594570,Заклад вищої освіти,Інститут,Приватна,...,КИЇВ,Київ,"вул. Фрометівська, 2",+380(44)-496-97-60,fb.mail.ir@gmail.com,www.ir.edu.ua,В. о. директора,Бойко Світлана Станіславівна,вищийнавчальнийзакладнститутдизайнуархтектурит...,80
4,"Вищий навчальний заклад ""Київська Академія пер...",865,NaN,"ВНЗ ""КАПМ""","Higher educational institution ""Kyiv Academy o...",2002,26199252,Заклад вищої освіти,Академія,Приватна,...,КИЇВ,Київ,"вул. Виборзька, 86",0444555672,kapm@ukr.net,www.kapi.com.ua,ректор,Лубянська Світлана Павлівна,вищийнавчальнийзакладкивськаакадемяперукарсько...,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,Відокремлений підрозділ національного універси...,2839,7,.,NaN,2000,35139007,Заклад вищої освіти,Відокремлений підрозділ,Державна,...,Чернігівська область,Бобровиця,"вул. Чернігівська, 19",NaN,NaN,NaN,NaN,NaN,вдокремленийпдроздлнацональногоунверситетуборе...,74
1416,Борзнянський державний сільськогосподарський т...,390,NaN,БДСГТ,State agricultural technical school of Borzna,1999,00729178,Заклад вищої освіти,Технікум (училище),Державна,...,Чернігівська область,Борзна,"Олекси Десняка, 23",21106,koba_22@ukr.net,bdsgt.at.ua,Виконуючий обов`язки директора,Линник Віталій Володимирович,борзнянськийдержавнийсльськогосподарськийтехнку,74
1417,Козелецький коледж ветеринарної медицини Білоц...,990,362,ККВМ БНАУ,"Kozelets College of Veterinary Medicine, Bila ...",1925,00729209,Заклад вищої освіти,Коледж,Державна,...,Чернігівська область,Козелець,"вул. Сім'ї Розумовських, 41","(046)462-12-00, (046)464-15-96, (046)462-14-83",techvet2@gmail.com,http://www.techvet.com.ua/,Директор коледжу,Редька Алла Іванівна,козелецькийколеджветеринарномедициниблоцерквсь...,74
1418,Остерський коледж будівництва та дизайну,925,NaN,ОКБД,Oster college of civil engineering and designing,1945,01355708,Заклад вищої освіти,Коледж,Державна,...,Чернігівська область,Остер,"вул. 8 Березня, 7-А",(046) 464-32-85,oster_budtex@cg.ukrtel.net,http://okbd.16mb.com,Директор,Бардаченко Тетяна Миколаївна,остерськийколеджбудвництватадизайн,74


In [96]:
univ_data_id[univ_data_id.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)]

,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,EDRPOU,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника",id,KOATUU_2020_reg


In [97]:
univ_data_id[(univ_data_id.EDRPOU.str.len() < 8)&(univ_data_id.EDRPOU.str.len().notna())]

,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,EDRPOU,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника",id,KOATUU_2020_reg


In [98]:
schools_ed_univ = merging(schools_ed_fin_new2, univ_data_id[['KOATUU_2020_reg', 'id', 'EDRPOU']], ['KOATUU_2020_reg', 'id'])
schools_ed_univ

Percentage matched: 51.2 %


,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU
0,Славгородська загальноосвітня школа I-II...,5922386401,"[2016, 2017]",59,славгородськазагальноосвтняшкола3ступкраснопль...,32758871
1,"Івашківський навчально-виховний комплекс ""Заг...",5122582601,[2016],51,вашквськийнавчальновиховнийкомплексзагальноосв...,
2,"Грабівський навчально-виховний комплекс ""Зага...",5122581601,[2016],51,грабвськийнавчальновиховнийкомплексзагальноосв...,
3,"Кримківська ордена ""Знак Пошани"" ЗОШ І-ІІІ ст...",4825482206,[2016],48,кримквськаорденазнакпошанизагальноосвтняшкола3...,
4,Лип'янська загальноосвітня школа І-ІІІ ступен...,7125784001,[2016],71,липянськазагальноосвтняшкола3ступшполянськорай...,24355707
...,...,...,...,...,...,...
27505,філія комунального опорного закладу освіти «Чо...,5123184201,[2019],51,флякомунальногоопорногозакладуосвтичорнянськаз...,
27506,"філія навчально-виховного комплексу ""Голованів...",3521482801,[2017],35,флянавчальновиховногокомплексуголованвськазага...,
27507,школа І-ІІІ ступенів № 104 імені О.Ольжича Обо...,8038000000,"[2018, 2019, 2020, 2021, 2022]",80,школа3ступ104меноольжичаоболонськогорайонумкив,22880852
27508,школа І-ІІІ ступенів №175 ім.В Марченка Шевчен...,8039100000,"[2020, 2021, 2022]",80,школа3ступ175менвмарченкашевченквськогорайонумкив,26125940


In [99]:
schools_ed_univ[schools_ed_univ['EDRPOU'] == ''].year.value_counts()

year
[2016]                            2563
[2022]                            1391
[2016, 2017]                      1362
[2016, 2017, 2018]                 925
[2021, 2022]                       840
                                  ... 
[2016, 2019, 2020, 2021, 2022]       1
[2017, 2020, 2021, 2022]             1
[2016, 2017, 2018, 2021, 2022]       1
[2018, 2022]                         1
[2016, 2018, 2020, 2022]             1
Name: count, Length: 88, dtype: int64

## коротка назва

In [100]:
univ_data_id.columns

Index(['Назва закладу освіти', 'Код', 'Код головного закладу', 'Коротка назва',
       'Назва закладу освіти (англ.)', 'Рік заснування', 'EDRPOU',
       'Вид закладу освіти', 'Тип закладу освіти', 'Форма власності',
       'Найменування органу, до сфери управління якого належить заклад освіти',
       'Поштовий індекс (фактична)', 'Код КОАТУУ (фактична)',
       'Регіон (фактична)', 'Населений пункт (фактична)', 'Адреса (фактична)',
       'Поштовий індекс (юридична)', 'KOATUU_2020', 'Регіон (юридична)',
       'Населений пункт (юридична)', 'Адреса (юридична)', 'Телефон / факс',
       'Електронна пошта', 'Веб-сайт', 'Посада керівника',
       'Прізвище, ім'я, по батькові керівника', 'id', 'KOATUU_2020_reg'],
      dtype='object')

In [101]:
univ_data_id_short = univ_data_id[univ_data_id['Коротка назва'].notna()]

In [102]:
create_id(univ_data_id_short, 'Коротка назва', 'id')

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/2540449143.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['id'] = dataset['id'].apply(lambda x: x[:-1])


,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,EDRPOU,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника",id,KOATUU_2020_reg
0,"Академія праці, соціальних відносин і туризму",53,NaN,АПСВТ,"Academy of Labour, Social Relations and Tourism",1993,04641405,Заклад вищої освіти,Академія,Приватна,...,КИЇВ,Київ,"вул. Велика Окружна дорога, 3",+38(044) 522-49-40,info@socosvita.kiev.ua,http://www. socosvita.kiev.ua,Ректор,Буяшенко Вікторія Василівна,апсв,80
1,Бізнес-коледж Приватного вищого навчального за...,3555,228,БК ПВНЗ «Європейський університет»,Business College of Private Higher Educational...,2002,40487422,Заклад вищої освіти,Коледж,Приватна,...,КИЇВ,Київ,"бульв. Академіка Вернадського, 16-В",+380(44)-432-00-58,bkeu@kyiv.e-u.in.ua,www.e-u.in.ua,директор,Сильницький Юрій Вікторович,бкпвнзвропейськийунверситет,80
2,"Вищий навчальний заклад ""Відкритий міжнародний...",217,NaN,"Університет ""Україна""",Higher Education Institution 'Open Internation...,1998,30373644,Заклад вищої освіти,Університет,Приватна,...,КИЇВ,Київ,"вул. Хорива, 1Г",(044) 4287410; 4252322; 4225019; 4252926,viddilorgrobotu@ukr.net,www.vmurol.com.ua,президент,Таланчук Петро Михайлович,унверситетукрана,80
3,"Вищий навчальний заклад Інститут дизайну, архі...",1319,NaN,"Вищий навчальний заклад ""Інститут дизайну, арх...",Higher educational institution “Institute of D...,2000,32594570,Заклад вищої освіти,Інститут,Приватна,...,КИЇВ,Київ,"вул. Фрометівська, 2",+380(44)-496-97-60,fb.mail.ir@gmail.com,www.ir.edu.ua,В. о. директора,Бойко Світлана Станіславівна,вищийнавчальнийзакладнститутдизайнуархтектурит...,80
4,"Вищий навчальний заклад ""Київська Академія пер...",865,NaN,"ВНЗ ""КАПМ""","Higher educational institution ""Kyiv Academy o...",2002,26199252,Заклад вищої освіти,Академія,Приватна,...,КИЇВ,Київ,"вул. Виборзька, 86",0444555672,kapm@ukr.net,www.kapi.com.ua,ректор,Лубянська Світлана Павлівна,внзкапм,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,Відокремлений підрозділ національного універси...,2839,7,.,NaN,2000,35139007,Заклад вищої освіти,Відокремлений підрозділ,Державна,...,Чернігівська область,Бобровиця,"вул. Чернігівська, 19",NaN,NaN,NaN,NaN,NaN,,74
1416,Борзнянський державний сільськогосподарський т...,390,NaN,БДСГТ,State agricultural technical school of Borzna,1999,00729178,Заклад вищої освіти,Технікум (училище),Державна,...,Чернігівська область,Борзна,"Олекси Десняка, 23",21106,koba_22@ukr.net,bdsgt.at.ua,Виконуючий обов`язки директора,Линник Віталій Володимирович,бдсг,74
1417,Козелецький коледж ветеринарної медицини Білоц...,990,362,ККВМ БНАУ,"Kozelets College of Veterinary Medicine, Bila ...",1925,00729209,Заклад вищої освіти,Коледж,Державна,...,Чернігівська область,Козелець,"вул. Сім'ї Розумовських, 41","(046)462-12-00, (046)464-15-96, (046)462-14-83",techvet2@gmail.com,http://www.techvet.com.ua/,Директор коледжу,Редька Алла Іванівна,кквмбна,74
1418,Остерський коледж будівництва та дизайну,925,NaN,ОКБД,Oster college of civil engineering and designing,1945,01355708,Заклад вищої освіти,Коледж,Державна,...,Чернігівська область,Остер,"вул. 8 Березня, 7-А",(046) 464-32-85,oster_budtex@cg.ukrtel.net,http://okbd.16mb.com,Директор,Бардаченко Тетяна Миколаївна,окб,74


In [103]:
univ_data_id_short[univ_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)]

,Назва закладу освіти,Код,Код головного закладу,Коротка назва,Назва закладу освіти (англ.),Рік заснування,EDRPOU,Вид закладу освіти,Тип закладу освіти,Форма власності,...,Регіон (юридична),Населений пункт (юридична),Адреса (юридична),Телефон / факс,Електронна пошта,Веб-сайт,Посада керівника,"Прізвище, ім'я, по батькові керівника",id,KOATUU_2020_reg
5,"Вищий навчальний заклад ""Київський економічний...",251,NaN,КЕІМ,"Higher Educational Institution ""Kyiv Economic ...",1992,16459143,Заклад вищої освіти,Інститут,Приватна,...,КИЇВ,Київ,"вул. Смоленська , 31-33",(044) 456-88-02 (044) 456-72-64,keimkiev@ukr.net,http://ecoman-university.kiev.ua/,Ректор,Рожок Володимир Давидович,ке,80
13,Відокремлений структурний підрозділ Київський ...,3038,127,.,Separated structural subdivision Kyiv Industri...,2015,39867890,Заклад вищої освіти,Коледж,Державна,...,КИЇВ,Київ,"вул. Новоукраїнська, 24-А","(044) 4562028, 4584515",indkit1@bigmir.net,www.kink-knuba.kiev.sch.in.ua,Директор,Кікто Світлана Михайлівна,,80
16,Воєнно-дипломатична академія імені Євгенія Бер...,2899,NaN,.,Military Diplomatic Academy named after Yevhen...,1994,14303342,Заклад вищої освіти,Академія,Державна,...,КИЇВ,Київ,"вул. Юрія Іллєнка, 81",(044) 483-15-64,vdamo@i.ua,NaN,Начальник Воєнно-дипломатичної академії імені ...,Крисяк Павло Васильович,,80
19,"Державна наукова установа ""Науково-практичний ...",3718,NaN,.,"State Institution of Science ""Research and Pra...",2009,05415786,Наукові інститути (установи),Інша наукова установа (організація),Державна,...,КИЇВ,Київ,"вул. Верхня, 5","044 2848453, 2846536, 2864700",pruemnaya.durektora@ukr.net,NaN,Директор,Дячук Дмитро Дмитрович,,80
37,"Державний вищий навчальний заклад ""Київський е...",453,NaN,КЕК,"Higher state educational establishment ""Kyiv E...",1956,04627049,Заклад вищої освіти,Коледж,Державна,...,КИЇВ,Київ,"вул. Івана Сергієнка, 7","(044) 558 17 72, ф. 559 38 81",contact@kek.kiev.ua,www.kek.edu.ua,директор,Левченко Михайло Іванович,ке,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,Сторожинецький коледж Вищого навчального закла...,1098,217,.,Storozhynets college Higher Education Institut...,2008,33304052,Заклад вищої освіти,Коледж,Приватна,...,Чернівецька область,Сторожинець,"вул. Клинівська, 20 корпус Б",0373525630,st-koledg@ukr.net,http://stor-coledj.com.ua,Директор,Горюк Павло Дмитрович,,73
1389,Сторожинецький навчально-науковий інститут Уні...,3641,3457,.,NaN,2000,40626727,Заклад вищої освіти,Інститут,Державна,...,Чернівецька область,Сторожинець,"вул. Видинівського, 3","(03735) 22857, 25479",37.06@nusta.edu.ua,http://www.nusta.edu.ua/,Директор,Гордіца Тетяна Маноліївна,,73
1400,"Чернігівський інститут інформації, бізнесу і п...",1369,195,.,Chernigov istitute information business and la...,2000,25623216,Заклад вищої освіти,Інститут,Приватна,...,Чернігівська область,Чернігів,"просп. Перемоги, 95-А",(0462) 671-099:(0462)670-150,chiibip.mnty@gmail.ru,NaN,В.о. ректора,Бобришев Євгеній Сергійович,,74
1411,Прилуцьке медичне училище,721,NaN,.,Pryluky Medical School,1927,02011700,Заклад вищої освіти,Технікум (училище),Комунальна,...,Чернігівська область,Прилуки,"вул. Київська, 243",(04637) 3-40-76,prilukimed@ukr.net,http://pmu.at.ua,Директор,Котляр Тетяна Іванівна,,74


In [104]:
univ_data_id_short.drop(univ_data_id_short[univ_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)].index.values.tolist()  , inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/2203993389.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  univ_data_id_short.drop(univ_data_id_short[univ_data_id_short.duplicated(subset=['id', 'KOATUU_2020_reg'], keep=False)].index.values.tolist()  , inplace=True)


In [105]:
schools_ed_univ = merging(schools_ed_univ, univ_data_id_short[['KOATUU_2020_reg', 'id', 'EDRPOU']], ['KOATUU_2020_reg', 'id'])
schools_ed_univ

Percentage matched: 51.3 %


,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU
0,Славгородська загальноосвітня школа I-II...,5922386401,"[2016, 2017]",59,славгородськазагальноосвтняшкола3ступкраснопль...,32758871
1,"Івашківський навчально-виховний комплекс ""Заг...",5122582601,[2016],51,вашквськийнавчальновиховнийкомплексзагальноосв...,
2,"Грабівський навчально-виховний комплекс ""Зага...",5122581601,[2016],51,грабвськийнавчальновиховнийкомплексзагальноосв...,
3,"Кримківська ордена ""Знак Пошани"" ЗОШ І-ІІІ ст...",4825482206,[2016],48,кримквськаорденазнакпошанизагальноосвтняшкола3...,
4,Лип'янська загальноосвітня школа І-ІІІ ступен...,7125784001,[2016],71,липянськазагальноосвтняшкола3ступшполянськорай...,24355707
...,...,...,...,...,...,...
27505,філія комунального опорного закладу освіти «Чо...,5123184201,[2019],51,флякомунальногоопорногозакладуосвтичорнянськаз...,
27506,"філія навчально-виховного комплексу ""Голованів...",3521482801,[2017],35,флянавчальновиховногокомплексуголованвськазага...,
27507,школа І-ІІІ ступенів № 104 імені О.Ольжича Обо...,8038000000,"[2018, 2019, 2020, 2021, 2022]",80,школа3ступ104меноольжичаоболонськогорайонумкив,22880852
27508,школа І-ІІІ ступенів №175 ім.В Марченка Шевчен...,8039100000,"[2020, 2021, 2022]",80,школа3ступ175менвмарченкашевченквськогорайонумкив,26125940


# collected with bot

In [106]:
data_bot = pd.read_csv('/Users/scipyguru/Downloads/nums.csv', names = ['eoname', 'url', 'EDRPOU'], dtype = str)
data_bot

,eoname,url,EDRPOU
0,"Тарасівський навчально-виховний комплекс ""Зага...",https://opendatabot.ua/c/33329179,33329179
1,"Великопільський навчально-виховний комплекс ""з...",https://opendatabot.ua/c/22398776,22398776
2,Лоївська загальноосвітня школа І-ІІІ ступенів ...,https://opendatabot.ua/c/20567335,20567335
3,"Хоровецький навчально- виховний комплекс ""дошк...",https://opendatabot.ua/c/22787615,Not found.
4,"Комунальний заклад освіти ""Вечірня (змінна) се...",https://opendatabot.ua/c/25725106,25725106
...,...,...,...
11874,Зорянський заклад загальної середньої освіти і...,https://opendatabot.ua/c/25708154,Not found.
11875,Стратіївський заклад загальної середньої освіт...,https://opendatabot.ua/c/26285678,Not found.
11876,Бібрський опорний ліцей імені Уляни Кравченко ...,https://opendatabot.ua/c/22368947,Not found.
11877,Литовезький ліцей імені Володимира Якобчука Во...,https://opendatabot.ua/c/23016593,Not found.


In [107]:
data_bot[(data_bot.EDRPOU.str.len() < 8)&(data_bot.EDRPOU.str.len().notna())]

,eoname,url,EDRPOU
87,Комунальний спеціалізований навчальний заклад ...,https://opendatabot.ua/c/04544524,4544524
157,Великочернігівська загальноосвітня школа I-III...,https://opendatabot.ua/c/06670629,6670629
226,Киківська загальноосвітня школа I-III ступенів,https://opendatabot.ua/c/06669773,6669773
231,"Комунальний заклад ""Навчально-виховний комплек...",https://opendatabot.ua/c/03874573,3874573
297,Чернігівська загальноосвітня школа-інтернат I-...,https://opendatabot.ua/c/05266346,5266346
...,...,...,...
11790,"Львівський фаховий коледж будівництва, архітек...",https://opendatabot.ua/c/01275963,1275963
11791,Комунальний заклад Львівської обласної ради Ль...,https://opendatabot.ua/c/02214745,2214745
11792,Комунальний заклад вищої освіти Львівської обл...,https://opendatabot.ua/c/01989036,1989036
11836,"Державна установа ""Ізмаїльський слідчий ізолятор""",https://opendatabot.ua/c/08564179,8564179


In [108]:
data_bot['EDRPOU']=data_bot['EDRPOU'].apply(lambda x: '0'+x if (isinstance(x, str) and 0<len(x)<8) else x)

In [109]:
data_bot['EDRPOU']=data_bot['EDRPOU'].apply(lambda x: '0'+x if (isinstance(x, str) and 0<len(x)<8) else x)

In [110]:
data_bot = data_bot[data_bot.EDRPOU.str.len() == 8]

In [111]:
data_bot.drop_duplicates(inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/2071075787.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bot.drop_duplicates(inplace=True)


In [112]:
data_bot[data_bot.duplicated(subset=['eoname'], keep=False)]

,eoname,url,EDRPOU
365,Киянська загальноосвітня школа I-III ступенів,https://opendatabot.ua/c/06669767,06669767
366,Киянська загальноосвітня школа I-III ступенів,https://opendatabot.ua/c/22053907,22053907
1934,Вільшанська загальноосвітня школа I-III ступенів,https://opendatabot.ua/c/23231802,23231802
1936,Вільшанська загальноосвітня школа I-III ступенів,https://opendatabot.ua/c/24022041,24022041


In [113]:
data_bot.drop(data_bot[data_bot.duplicated(subset=['eoname'], keep=False)].index.values.tolist(), inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/3085644636.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bot.drop(data_bot[data_bot.duplicated(subset=['eoname'], keep=False)].index.values.tolist(), inplace=True)


In [114]:
schools_final = merging(schools_ed_univ, data_bot[['eoname', 'EDRPOU']], ['eoname'])
schools_final

Percentage matched: 74.8 %


,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU
0,Славгородська загальноосвітня школа I-II...,5922386401,"[2016, 2017]",59,славгородськазагальноосвтняшкола3ступкраснопль...,32758871
1,"Івашківський навчально-виховний комплекс ""Заг...",5122582601,[2016],51,вашквськийнавчальновиховнийкомплексзагальноосв...,
2,"Грабівський навчально-виховний комплекс ""Зага...",5122581601,[2016],51,грабвськийнавчальновиховнийкомплексзагальноосв...,
3,"Кримківська ордена ""Знак Пошани"" ЗОШ І-ІІІ ст...",4825482206,[2016],48,кримквськаорденазнакпошанизагальноосвтняшкола3...,
4,Лип'янська загальноосвітня школа І-ІІІ ступен...,7125784001,[2016],71,липянськазагальноосвтняшкола3ступшполянськорай...,24355707
...,...,...,...,...,...,...
27505,філія комунального опорного закладу освіти «Чо...,5123184201,[2019],51,флякомунальногоопорногозакладуосвтичорнянськаз...,
27506,"філія навчально-виховного комплексу ""Голованів...",3521482801,[2017],35,флянавчальновиховногокомплексуголованвськазага...,
27507,школа І-ІІІ ступенів № 104 імені О.Ольжича Обо...,8038000000,"[2018, 2019, 2020, 2021, 2022]",80,школа3ступ104меноольжичаоболонськогорайонумкив,22880852
27508,школа І-ІІІ ступенів №175 ім.В Марченка Шевчен...,8039100000,"[2020, 2021, 2022]",80,школа3ступ175менвмарченкашевченквськогорайонумкив,26125940


In [115]:
schools_final = schools_final.sort_values(by=['KOATUU_2020', 'eoname'])

In [116]:
data_bot1 = pd.read_csv('/Users/scipyguru/Downloads/nums (1).csv', names = ['eoname', 'url', 'EDRPOU'], dtype = str)
data_bot1

,eoname,url,EDRPOU
0,"ВСП ""Технологічно-промисловий фаховий коледж В...",https://opendatabot.ua/c/00419667,Not found.
1,Вище професійне училище №11 м. Вінниці,https://opendatabot.ua/c/03065891,3065891
2,Вище художнє професійно-технічне училище №5 м....,https://opendatabot.ua/c/02539890,Not found.
3,Відокремлений структурний підрозділ «Вінницьки...,https://opendatabot.ua/c/01565891,Not found.
4,Відокремлений структурний підрозділ Вінницький...,https://opendatabot.ua/c/01355656,Not found.
...,...,...,...
5867,Школа №175 І-ІІІ ступенів Шевченківського райо...,https://opendatabot.ua/c/26125940,Not found.
5868,Школа №199 І-ІІІ ступенів Шевченківського райо...,https://opendatabot.ua/c/22881923,Not found.
5869,Школа №27 І-ІІІ ступенів Шевченківського район...,https://opendatabot.ua/c/26125710,Not found.
5870,Школа №70 І-ІІІ ступенів Шевченківського район...,https://opendatabot.ua/c/22881828,Not found.


In [117]:
data_bot1 = data_bot1[data_bot1.EDRPOU.str.len() == 8]

In [118]:
data_bot1.drop_duplicates(inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/2420936744.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_bot1.drop_duplicates(inplace=True)


In [119]:
data_bot1[data_bot1.duplicated(subset=['eoname'], keep=False)]

,eoname,url,EDRPOU


In [120]:
schools_final1 = merging(schools_final, data_bot1[['eoname', 'EDRPOU']], ['eoname'])
schools_final1

Percentage matched: 74.9 %


,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU
0,"ВСП ""Технологічно-промисловий фаховий коледж В...",0510100000,[2022],05,вдокремленийструктурнийпдроздлтехнологчнопроми...,
1,Вище професійне училище №11 м. Вінниці,0510100000,"[2018, 2019, 2020, 2021, 2022]",05,вищепрофесйнеучилище11мвниц,03065891
2,Вище художнє професійно-технічне училище №5 м....,0510100000,"[2018, 2019, 2020, 2021, 2022]",05,вищехудожнпрофесйнотехнчнеучилище5мвниц,
3,"Відокремлений структурний підрозділ ""Вінницьки...",0510100000,"[2021, 2022]",05,вдокремленийструктурнийпдроздлвницькийколеджна...,35725833
4,"Відокремлений структурний підрозділ ""Вінницьки...",0510100000,"[2021, 2022]",05,вдокремленийструктурнийпдроздлвницькийколеджек...,38417484
...,...,...,...,...,...,...
27505,Школа №25 І-ІІІ ступенів Шевченківського район...,8039100000,"[2016, 2017, 2018, 2019, 2020, 2021]",80,школа253ступшевченквськогорайонумкив,
27506,Школа №27 І-ІІІ ступенів Шевченківського район...,8039100000,[2016],80,школа273ступшевченквськогорайонумкив,
27507,Школа №70 І-ІІІ ступенів Шевченківського район...,8039100000,[2016],80,школа703ступшевченквськогорайонумкив,
27508,Школа №95 І-ІІІ ступенів Шевченківського район...,8039100000,[2017],80,школа953ступшевченквськогорайонумкив,


# Youcontrol

In [121]:
youcontrol =  pd.read_excel('/Users/scipyguru/Downloads/Навчальні-заклади-історичні-назви.xlsx', dtype='str')
youcontrol 

,Код ЄДРПОУ,Назва,Коротка назва
0,00121146,"ВІДКРИТЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...",NaN
1,00121146,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ"""
2,00121146,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ"""
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN
4,00131446,ІНШІ ОРГАНІЗАЦІЙНО-ПРАВОВІ ФОРМИ КОЛЕКТИВНЕ ПІ...,NaN
...,...,...,...
80921,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""
80923,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""
80924,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ"""


In [122]:
youcontrol.columns

Index(['Код ЄДРПОУ', 'Назва', 'Коротка назва'], dtype='object')

## long name

In [123]:
create_id(youcontrol, 'Назва', 'id')

,Код ЄДРПОУ,Назва,Коротка назва,id
0,00121146,"ВІДКРИТЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...",NaN,вдкритеакцонернетовариствопвдентеплоенергомонтаж
1,00121146,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",публчнеакцонернетовариствопвдентеплоенергомонтаж
2,00121146,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",приватнеакцонернетовариствопвдентеплоенергомонтаж
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN,колективнепдпримствоблгородднстровськамеханзов...
4,00131446,ІНШІ ОРГАНІЗАЦІЙНО-ПРАВОВІ ФОРМИ КОЛЕКТИВНЕ ПІ...,NaN,ншорганзацйноправовформиколективнепдпримствобл...
...,...,...,...,...
80921,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорнийзакладзагальносередньоосвт...
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинськийзакладзагальносередн...
80923,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинськийзакладзагальносередн...
80924,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинськийзакладзагальносередн...


In [124]:
youcontrol.rename(columns={'Код ЄДРПОУ':'EDRPOU'}, inplace=True)

In [125]:
youcontrol[youcontrol.EDRPOU == '33155713'].id.unique()

array(['товариствозобмеженоювдповдальнстюфранцузькашколамкива',
       'товариствозобмеженоювдповдальнстюфранцузькийлцейменаникивсько'],
      dtype=object)

In [126]:
youcontrol[(youcontrol.EDRPOU.str.len() < 8)&(youcontrol.EDRPOU.str.len().notna())]

,EDRPOU,Назва,Коротка назва,id


In [127]:
youcontrol[youcontrol.duplicated(subset=['id', 'EDRPOU'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
3,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",NaN,колективнепдпримствоблгородднстровськамеханзов...
5,00131446,"КОЛЕКТИВНЕ ПІДПРИЄМСТВО ""БІЛГОРОД-ДНІСТРОВСЬКА...",КП МК-26,колективнепдпримствоблгородднстровськамеханзов...
67,00290280,"ВІДКРИТЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""НОВООЛЕКСАНДРІ...",NaN,вдкритеакцонернетовариствоновоолександрвськийц...
68,00290280,"ВІДКРИТЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""НОВОOЛЕКСАНДРІ...",NaN,вдкритеакцонернетовариствоновоолександрвськийц...
119,00452713,"ДЕРЖАВНИЙ НАВЧАЛЬНИЙ ЗАКЛАД ""ЗВЕНИГОРОДСЬКИЙ Ц...","ДНЗ ""ЗВЕНИГОРОДСЬКИЙ ЦППРК""",державнийнавчальнийзакладзвенигородськийцентрп...
...,...,...,...,...
80920,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорнийзакладзагальносередньоосвт...
80921,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,деребчинськийопорнийзакладзагальносередньоосвт...
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинськийзакладзагальносередн...
80923,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинськийзакладзагальносередн...


In [128]:
youcontrol = youcontrol.drop_duplicates(subset=['id', 'EDRPOU'], keep='first')

In [129]:
youcontrol[youcontrol.duplicated(subset=['id'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
221,01284979,МІЖШКІЛЬНИЙ НАВЧАЛЬНО-ВИРОБНИЧИЙ КОМБІНАТ,МНВК,мжшкльнийнавчальновиробничийкомбна
298,01562680,ПРОФЕСІЙНО-ТЕХНІЧНЕ УЧИЛИЩЕ № 41,NaN,професйнотехнчнеучилище4
304,01566423,ПРОФЕСІЙНО-ТЕХНІЧНЕ УЧИЛИЩЕ № 24,NaN,професйнотехнчнеучилище2
309,01566488,"ДЕРЖАВНА ОРГАНІЗАЦІЯ (УСТАНОВА, ЗАКЛАД) ПРОФЕС...",NaN,державнаорганзацяустановазакладпрофесйнотехнчн...
317,01566502,"ДЕРЖАВНА ОРГАНІЗАЦІЯ (УСТАНОВА, ЗАКЛАД) ПРОФЕС...",NaN,державнаорганзацяустановазакладпрофесйнотехнчн...
...,...,...,...,...
80472,43990324,"МІЖНАРОДНА АКАДЕМІЧНА ШКОЛА ""ОДЕСА""","МАШ ""ОДЕСА""",мжнароднаакадемчнашколаодеса
80903,45251511,Юріївська гімназія з початковою школою та дошк...,Юріївська гімназія,юрвськагмназязпочатковоюшколоютадошкльнимпдроз...
80905,45275530,Юріївська гімназія з початковою школою та дошк...,Юріївська гімназія,юрвськагмназязпочатковоюшколоютадошкльнимпдроз...
80907,45287518,Кобзарцівська початкова школа з дошкільним під...,Кобзарцівська початкова школа,кобзарцвськапочатковашколаздошкльнимпдроздломс...


In [130]:
youcontrol.drop(youcontrol[youcontrol.duplicated(subset=['id'], keep=False)].index.values.tolist()  , inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/1258219814.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youcontrol.drop(youcontrol[youcontrol.duplicated(subset=['id'], keep=False)].index.values.tolist()  , inplace=True)


In [131]:
schools_final2 = merging(schools_final1, youcontrol[['id', 'EDRPOU']], ['id'])
schools_final2

Percentage matched: 83.8 %


,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU
0,"ВСП ""Технологічно-промисловий фаховий коледж В...",0510100000,[2022],05,вдокремленийструктурнийпдроздлтехнологчнопроми...,
1,Вище професійне училище №11 м. Вінниці,0510100000,"[2018, 2019, 2020, 2021, 2022]",05,вищепрофесйнеучилище11мвниц,03065891
2,Вище художнє професійно-технічне училище №5 м....,0510100000,"[2018, 2019, 2020, 2021, 2022]",05,вищехудожнпрофесйнотехнчнеучилище5мвниц,02539890
3,"Відокремлений структурний підрозділ ""Вінницьки...",0510100000,"[2021, 2022]",05,вдокремленийструктурнийпдроздлвницькийколеджна...,35725833
4,"Відокремлений структурний підрозділ ""Вінницьки...",0510100000,"[2021, 2022]",05,вдокремленийструктурнийпдроздлвницькийколеджек...,38417484
...,...,...,...,...,...,...
27505,Школа №25 І-ІІІ ступенів Шевченківського район...,8039100000,"[2016, 2017, 2018, 2019, 2020, 2021]",80,школа253ступшевченквськогорайонумкив,
27506,Школа №27 І-ІІІ ступенів Шевченківського район...,8039100000,[2016],80,школа273ступшевченквськогорайонумкив,
27507,Школа №70 І-ІІІ ступенів Шевченківського район...,8039100000,[2016],80,школа703ступшевченквськогорайонумкив,
27508,Школа №95 І-ІІІ ступенів Шевченківського район...,8039100000,[2017],80,школа953ступшевченквськогорайонумкив,


## short name


In [132]:
youcontrol = youcontrol[youcontrol['Коротка назва'].notna()]

In [133]:
create_id(youcontrol, 'Коротка назва', 'id')

,EDRPOU,Назва,Коротка назва,id
1,00121146,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",атптем
2,00121146,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",атптем
7,00131564,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПРИКАРПАТТЯОБЛ...","ПАТ ""ПРИКАРПАТТЯОБЛЕНЕРГО""",патприкарпаттяобленерго
8,00131564,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПРИКАРПАТТЯОБЛ...","АТ ""ПРИКАРПАТТЯОБЛЕНЕРГО""",атприкарпаттяобленерго
11,00131954,"АКЦІОНЕРНА ТОВАРИСТВО ""ХАРКІВОБЛЕНЕРГО""","АТ ""ХАРКІВОБЛЕНЕРГО""",атхарквобленерго
...,...,...,...,...
80913,45385954,ДУБРІВСЬКА ГІМНАЗІЯ СЕРЕДНЯНСЬКОЇ СЕЛИЩНОЇ РАД...,ДУБРІВСЬКА ГІМНАЗІЯ,дубрвськагмназ
80914,45407202,"ФЕРМЕРСЬКЕ ГОСПОДАРСТВО ""Зеленогірське""","ФГ ""Зеленогірське""",фгзеленогрське
80918,60043961,ДЕРЕБЧИНСЬКИЙ ОПОРНИЙ ЗАКЛАД ЗАГАЛЬНОЇ СЕРЕДНЬ...,ДОЗЗСО,доззс
80922,60043961,"ОПОРНИЙ ЗАКЛАД ""ДЕРЕБЧИНСЬКИЙ ЗАКЛАД ЗАГАЛЬНОЇ...","ОЗ ""ДЕРЕБЧИНСЬКИЙ ЗЗСО I-III СТУПЕНІВ""",опорнийзакладдеребчинськийзакладзагальносередн...


In [134]:
youcontrol[youcontrol.duplicated(subset=['id', 'EDRPOU'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
1,00121146,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",атптем
2,00121146,"ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""ПІВДЕНТЕПЛОЕНЕ...","АТ ""ПТЕМ""",атптем
11,00131954,"АКЦІОНЕРНА ТОВАРИСТВО ""ХАРКІВОБЛЕНЕРГО""","АТ ""ХАРКІВОБЛЕНЕРГО""",атхарквобленерго
12,00131954,"АКЦІОНЕРНЕ ТОВАРИСТВО ""ХАРКІВОБЛЕНЕРГО""","АТ ""ХАРКІВОБЛЕНЕРГО""",атхарквобленерго
62,00220434,"ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО ""НАУКОВО-ВИРОБН...","АТ ""НВАТ ""ВНДІКОМПРЕСОРМАШ""",атнватвндкомпресормаш
...,...,...,...,...
80821,44881222,"Комунальний заклад ""Гладківщинська гімназія Пі...","Комунальний заклад ""Гладківщинська гімназія Пі...",комунальнийзакладгладквщинськагмназяпщанськосл...
80842,44952080,"""ЦЕНТР НАДАННЯ СОЦІАЛЬНИХ ПОСЛУГ"" ВЕЛИКОМИХАЙЛ...","КУ ""ЦНСП"" ВЕЛИКОМИХАЙЛІВСЬКОЇ СЕЛИЩНОЇ РАДИ",куцнспвеликомихайлвськоселищнорад
80844,44952080,"КОМУНАЛЬНА УСТАНОВА ""ЦЕНТР НАДАННЯ СОЦІАЛЬНИХ ...","КУ ""ЦНСП"" ВЕЛИКОМИХАЙЛІВСЬКОЇ СЕЛИЩНОЇ РАДИ",куцнспвеликомихайлвськоселищнорад
80895,45145609,ПЕРВИННА ПРОФСПІЛКОВА ОРГАНІЗАЦІЯ ДЕРЖАВНОГО Н...,"ППО ДНЗ ""СЦПТОХТТРС""",пподнзсцптохттрс


In [135]:
youcontrol = youcontrol.drop_duplicates(subset=['id', 'EDRPOU'], keep='first')

In [136]:
youcontrol[youcontrol.duplicated(subset=['id'], keep=False)]

,EDRPOU,Назва,Коротка назва,id
194,01116408,Стрийська початкова школа №12 Стрийської міськ...,Стрийська початкова школа №12,стрийськапочатковашкола1
271,01488943,ШКОЛА I-III СТУПЕНІВ № 292 ІМЕНІ ГЕТЬМАНА УКРА...,ШКОЛА № 292,школа29
279,01489305,ШКОЛА І-ІІІ СТУПЕНІВ № 294 ДЕСНЯНСЬКОГО РАЙОН...,ШКОЛА № 294,школа29
295,01489345,ЛІЦЕЙ №18 ДЕСНЯНСЬКОГО РАЙОНУ МІСТА КИЄВА,ЛІЦЕЙ №18,лцей1
332,01730137,ОЛЕКСАНДРІВСЬКИЙ КОМБІНАТ КООПЕРАТИВНОЇ ПРОМИС...,ОЛЕКСАНДРІВСЬКИЙ ККП,олександрвськийкк
...,...,...,...,...
80763,44768762,МИХАЙЛІВСЬКА ГІМНАЗІЯ ВЕСЕЛИНІВСЬКОЇ СЕЛИЩНОЇ ...,МИХАЙЛІВСЬКА ГІМНАЗІЯ,михайлвськагмназ
80799,44845045,КРАСНОПІЛЬСЬКА ГІМНАЗІЯ ВРАДІЇВСЬКОЇ СЕЛИЩНОЇ ...,КРАСНОПІЛЬСЬКА ГІМНАЗІЯ,краснопльськагмназ
80811,44869019,ФЕДОРІВСЬКА ГІМНАЗІЯ ВРАДІЇВСЬКОЇ СЕЛИЩНОЇ РАД...,ФЕДОРІВСЬКА ГІМНАЗІЯ,федорвськагмназ
80882,45097959,Боришковецька гімназія (зі структурними підроз...,Боришковецька гімназія,боришковецькагмназ


In [137]:
youcontrol.drop(youcontrol[youcontrol.duplicated(subset=['id'], keep=False)].index.values.tolist()  , inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/1258219814.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youcontrol.drop(youcontrol[youcontrol.duplicated(subset=['id'], keep=False)].index.values.tolist()  , inplace=True)


In [138]:
schools_final3 = merging(schools_final2, youcontrol[['id', 'EDRPOU']], ['id'])
schools_final3

Percentage matched: 84.1 %


,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU
0,"ВСП ""Технологічно-промисловий фаховий коледж В...",0510100000,[2022],05,вдокремленийструктурнийпдроздлтехнологчнопроми...,
1,Вище професійне училище №11 м. Вінниці,0510100000,"[2018, 2019, 2020, 2021, 2022]",05,вищепрофесйнеучилище11мвниц,03065891
2,Вище художнє професійно-технічне училище №5 м....,0510100000,"[2018, 2019, 2020, 2021, 2022]",05,вищехудожнпрофесйнотехнчнеучилище5мвниц,02539890
3,"Відокремлений структурний підрозділ ""Вінницьки...",0510100000,"[2021, 2022]",05,вдокремленийструктурнийпдроздлвницькийколеджна...,35725833
4,"Відокремлений структурний підрозділ ""Вінницьки...",0510100000,"[2021, 2022]",05,вдокремленийструктурнийпдроздлвницькийколеджек...,38417484
...,...,...,...,...,...,...
27505,Школа №25 І-ІІІ ступенів Шевченківського район...,8039100000,"[2016, 2017, 2018, 2019, 2020, 2021]",80,школа253ступшевченквськогорайонумкив,
27506,Школа №27 І-ІІІ ступенів Шевченківського район...,8039100000,[2016],80,школа273ступшевченквськогорайонумкив,
27507,Школа №70 І-ІІІ ступенів Шевченківського район...,8039100000,[2016],80,школа703ступшевченквськогорайонумкив,
27508,Школа №95 І-ІІІ ступенів Шевченківського район...,8039100000,[2017],80,школа953ступшевченквськогорайонумкив,


# One school

In [139]:
schools_final4 = schools_final3.groupby(by=['KOATUU_2020']).agg(
    {
        "id": lambda x: len(x.unique()),
        "eoname": 'first',
        "EDRPOU": 'first'
    }
).reset_index()
schools_final4 = schools_final4[schools_final4['id'] == 1]
schools_final4 = schools_final4[schools_final4.EDRPOU == '']
schools_final4 = schools_final4[['KOATUU_2020','eoname']]
schools_final4.head()

,KOATUU_2020,eoname
11,0520281401,"Заклад ""Гулівська загальноосвітня школа I-III ..."
143,0521686101,Комунальний заклад «Навчально-виховний комплек...
175,0522481603,Середня загальноосвітня школа I-III ступенів с...
185,0522486201,Середня загальноосвітня школа I-III ступенів с...
219,0523082001,Вишковецька загальноосвітня школа І-ІІІ ступен...


In [140]:
schools_data_one = schools_data_id.groupby(by=['KOATUU_2020']).agg(
    {
        "Повна назва": lambda x: len(x.unique()),
    }
).reset_index()

schools_data_one = pd.DataFrame(schools_data_one[schools_data_one['Повна назва'] == 1]['KOATUU_2020'])
schools_data_one.head()

,KOATUU_2020
5,0510690002
8,0520255300
9,0520280102
10,0520280201
11,0520280207


In [141]:
schools_data_one = schools_data_one.merge(schools_data_id, on='KOATUU_2020', how='inner')

In [142]:
schools_data_one = schools_data_one[['KOATUU_2020', 'Повна назва', 'EDRPOU']]
schools_data_one.head()

,KOATUU_2020,Повна назва,EDRPOU
0,0510690002,Лукашівський навчально-виховний комплекс Ладиж...,26219575
1,0520255300,"Комунальний заклад ""Копайгородський ліцей""",13326542
2,0520280102,"Глинянський навчально- виховний комплекс "" Заг...",26233629
3,0520280201,Балківський заклад загальної середньої освіти ...,13337681
4,0520280207,Окладнянський заклад загальної середньої освіт...,26243438


In [143]:
df_one = schools_final4.merge(schools_data_one, how='inner', left_on='KOATUU_2020', right_on='KOATUU_2020')
#df_one = df_one.drop(columns=['Код КОАТУУ'])
df_one

,KOATUU_2020,eoname,Повна назва,EDRPOU
0,0520281401,"Заклад ""Гулівська загальноосвітня школа I-III ...",Гулівський заклад загальної середньої освіти І...,26229480
1,0522481603,Середня загальноосвітня школа I-III ступенів с...,філія І-ІІ ступенів с. Вінниківці опорного зак...,26244432
2,0522486201,Середня загальноосвітня школа I-III ступенів с...,Філія І-ІІ ступенів с. Селище Літинського ліце...,44602060
3,0523082001,Вишковецька загальноосвітня школа І-ІІІ ступен...,Вишковецька гімназія Брацлавської селищної рад...,25918762
4,0523083601,Зяньковецька загальноосвітня школа І-ІІІ ступе...,Зяньковецька гімназія Брацлавської селищної ра...,26227988
...,...,...,...,...
133,7120680501,Безбородьківська загальноосвітня школа І-ІІІ с...,заклад загальної середньої освіти I-II ступені...,24353281
134,7122882401,"Вотилівський навчально-виховний комплекс ""Дошк...",Вотилівська філія комунального опорного заклад...,33327260
135,7122884001,"Кам'янобрідський навчально-виховний комплекс ""...","Філія "" Кам'янобрідська початкова школа з дошк...",33329755
136,7122887601,"Хижинський навчально-виховний комплекс ""Дошкіл...",Хижинська початкова школа - філія опорного зак...,33220394


In [144]:
df_one[df_one.duplicated(subset=['eoname'], keep=False)]

,KOATUU_2020,eoname,Повна назва,EDRPOU


In [145]:
df_one.drop(df_one[df_one.duplicated(subset=['eoname'], keep=False)].index.values.tolist()  , inplace=True)

In [146]:
schools_final4 = merging(schools_final3, df_one[['eoname', 'EDRPOU']], ['eoname'])
schools_final4

Percentage matched: 84.7 %


,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU
0,"ВСП ""Технологічно-промисловий фаховий коледж В...",0510100000,[2022],05,вдокремленийструктурнийпдроздлтехнологчнопроми...,
1,Вище професійне училище №11 м. Вінниці,0510100000,"[2018, 2019, 2020, 2021, 2022]",05,вищепрофесйнеучилище11мвниц,03065891
2,Вище художнє професійно-технічне училище №5 м....,0510100000,"[2018, 2019, 2020, 2021, 2022]",05,вищехудожнпрофесйнотехнчнеучилище5мвниц,02539890
3,"Відокремлений структурний підрозділ ""Вінницьки...",0510100000,"[2021, 2022]",05,вдокремленийструктурнийпдроздлвницькийколеджна...,35725833
4,"Відокремлений структурний підрозділ ""Вінницьки...",0510100000,"[2021, 2022]",05,вдокремленийструктурнийпдроздлвницькийколеджек...,38417484
...,...,...,...,...,...,...
27505,Школа №25 І-ІІІ ступенів Шевченківського район...,8039100000,"[2016, 2017, 2018, 2019, 2020, 2021]",80,школа253ступшевченквськогорайонумкив,
27506,Школа №27 І-ІІІ ступенів Шевченківського район...,8039100000,[2016],80,школа273ступшевченквськогорайонумкив,
27507,Школа №70 І-ІІІ ступенів Шевченківського район...,8039100000,[2016],80,школа703ступшевченквськогорайонумкив,
27508,Школа №95 І-ІІІ ступенів Шевченківського район...,8039100000,[2017],80,школа953ступшевченквськогорайонумкив,


# Gathered by hand

In [147]:
data_hand = pd.read_csv('/Users/scipyguru/Downloads/Final edrpou - schools_edrpou.csv',  dtype = str)
data_hand

,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU,Unnamed: 6
0,"ВСП ""Технологічно-промисловий фаховий коледж В...",0510100000,[2022],05,вдокремленийструктурнийпдроздлтехнологчнопроми...,00419667,https://youcontrol.com.ua/catalog/company_deta...
1,Вище професійне училище №11 м. Вінниці,0510100000,"[2018, 2019, 2020, 2021, 2022]",05,вищепрофесйнеучилище11мвниц,03065891,NaN
2,Вище художнє професійно-технічне училище №5 м....,0510100000,"[2018, 2019, 2020, 2021, 2022]",05,вищехудожнєпрофесйнотехнчнеучилище5мвниц,02539890,NaN
3,"Відокремлений структурний підрозділ ""Вінницьки...",0510100000,"[2021, 2022]",05,вдокремленийструктурнийпдроздлвницькийколеджна...,35725833,NaN
4,"Відокремлений структурний підрозділ ""Вінницьки...",0510100000,"[2021, 2022]",05,вдокремленийструктурнийпдроздлвницькийколеджек...,38417484,NaN
...,...,...,...,...,...,...,...
27505,Школа №25 І-ІІІ ступенів Шевченківського район...,8039100000,"[2016, 2017, 2018, 2019, 2020, 2021]",80,школа253ступшевченквськогорайонумкиєв,22880786,NaN
27506,Школа №27 І-ІІІ ступенів Шевченківського район...,8039100000,[2016],80,школа273ступшевченквськогорайонумкиєв,26125710,NaN
27507,Школа №70 І-ІІІ ступенів Шевченківського район...,8039100000,[2016],80,школа703ступшевченквськогорайонумкиєв,22881828,NaN
27508,Школа №95 І-ІІІ ступенів Шевченківського район...,8039100000,[2017],80,школа953ступшевченквськогорайонумкиєв,26125905,NaN


In [148]:
data_hand.columns

Index(['eoname', 'KOATUU_2020', 'year', 'KOATUU_2020_reg', 'id', 'EDRPOU',
       'Unnamed: 6'],
      dtype='object')

In [149]:
data_hand = data_hand[['eoname', 'KOATUU_2020', 'EDRPOU', 'KOATUU_2020_reg']]

In [150]:
data_hand[data_hand.duplicated(subset=['eoname', 'KOATUU_2020'], keep=False)]

,eoname,KOATUU_2020,EDRPOU,KOATUU_2020_reg
3310,Новомосковська загальноосвітня школа І-ІІІ сту...,1211900000,NaN,12
3311,Новомосковська загальноосвітня школа І-ІІІ сту...,1211900000,NaN,12
3457,Апостолівський районний ліцей - інтернат,1220310300,20281627,12
3458,Апостолівський районний ліцей - інтернат,1220310300,NaN,12
9897,Бучанська Українська гімназія,3210800000,24889099,32
9898,Бучанська Українська гімназія,3210800000,24889099,32
10667,Комунальний заклад Миронівська загальноосвітня...,3222910100,NaN,32
10668,Комунальний заклад Миронівська загальноосвітня...,3222910100,NaN,32
10669,Комунальний заклад Миронівська загальноосвітня...,3222910100,25302274,32
10670,Комунальний заклад Миронівська загальноосвітня...,3222910100,25302274,32


In [151]:
data_hand.drop(data_hand[data_hand.duplicated(subset=['eoname'], keep=False)].index.values.tolist()  , inplace=True)

/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_79204/426309281.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_hand.drop(data_hand[data_hand.duplicated(subset=['eoname'], keep=False)].index.values.tolist()  , inplace=True)


In [152]:
schools_final5 = merging(schools_final4, data_hand[['eoname', 'EDRPOU', 'KOATUU_2020']], ['eoname', 'KOATUU_2020'])
schools_final5

Percentage matched: 93.4 %


,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU
0,"ВСП ""Технологічно-промисловий фаховий коледж В...",0510100000,[2022],05,вдокремленийструктурнийпдроздлтехнологчнопроми...,00419667
1,Вище професійне училище №11 м. Вінниці,0510100000,"[2018, 2019, 2020, 2021, 2022]",05,вищепрофесйнеучилище11мвниц,03065891
2,Вище художнє професійно-технічне училище №5 м....,0510100000,"[2018, 2019, 2020, 2021, 2022]",05,вищехудожнпрофесйнотехнчнеучилище5мвниц,02539890
3,"Відокремлений структурний підрозділ ""Вінницьки...",0510100000,"[2021, 2022]",05,вдокремленийструктурнийпдроздлвницькийколеджна...,35725833
4,"Відокремлений структурний підрозділ ""Вінницьки...",0510100000,"[2021, 2022]",05,вдокремленийструктурнийпдроздлвницькийколеджек...,38417484
...,...,...,...,...,...,...
27505,Школа №25 І-ІІІ ступенів Шевченківського район...,8039100000,"[2016, 2017, 2018, 2019, 2020, 2021]",80,школа253ступшевченквськогорайонумкив,22880786
27506,Школа №27 І-ІІІ ступенів Шевченківського район...,8039100000,[2016],80,школа273ступшевченквськогорайонумкив,26125710
27507,Школа №70 І-ІІІ ступенів Шевченківського район...,8039100000,[2016],80,школа703ступшевченквськогорайонумкив,22881828
27508,Школа №95 І-ІІІ ступенів Шевченківського район...,8039100000,[2017],80,школа953ступшевченквськогорайонумкив,26125905


In [153]:
schools_final5[schools_final5.EDRPOU == '']

,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU
394,Степенівський навчально - виховний комплекс: з...,0520687603,[2016],05,степенвськийнавчальновиховнийкомплексзагальноо...,
697,Загальноосвітня школа I-III ступенів №2 с.Пиків,0521686303,"[2016, 2017, 2018, 2019]",05,загальноосвтняшкола3ступ2спик,
989,Загальноосвітня школа I-III ступенів с.Човнови...,0523186601,"[2018, 2019, 2020, 2021]",05,загальноосвтняшкола3ступсчовновицяоратвськогор...,
991,Загальноосвітня школа I-III ступенів с.Чернявк...,0523186901,"[2018, 2019, 2020, 2021]",05,загальноосвтняшкола3ступсчернявкаоратвськогора...,
995,"Навчально виховний комплекс ""Загальноосвітня ш...",0523255100,"[2016, 2017]",05,навчальновиховнийкомплексзагальноосвтняшкола3с...,
...,...,...,...,...,...,...
27312,Гімназія біотехнологій №177 Солом'янського рай...,8038900000,"[2016, 2017, 2018, 2019, 2020, 2021, 2022]",80,гмназяботехнологй177соломянськогорайонумкив,
27328,Київський спортивний ліцей-інтернат міського п...,8038900000,"[2016, 2017, 2018, 2019, 2020, 2021, 2022]",80,кивськийспортивнийлцейнтернатмськогопдпорядкуван,
27390,Технічний ліцей Національного технічного уніве...,8038900000,"[2016, 2017, 2018, 2019, 2020, 2021, 2022]",80,технчнийлцейнацональноготехнчногоунверситетуук...,
27479,"Товариство з обмеженою відповідальністю ""Франц...",8039100000,"[2016, 2017, 2018, 2019, 2020, 2021, 2022]",80,товариствозобмеженоювдповдальнстюфранцузькийлц...,


# End

In [154]:
schools_final5 = schools_final5.sort_values(by=['KOATUU_2020', 'eoname'])

In [156]:
schools_final5.to_csv('schools_edrpou.csv', index = False)

In [157]:
schools_final5[schools_final5.EDRPOU == '']

,eoname,KOATUU_2020,year,KOATUU_2020_reg,id,EDRPOU
394,Степенівський навчально - виховний комплекс: з...,0520687603,[2016],05,степенвськийнавчальновиховнийкомплексзагальноо...,
697,Загальноосвітня школа I-III ступенів №2 с.Пиків,0521686303,"[2016, 2017, 2018, 2019]",05,загальноосвтняшкола3ступ2спик,
989,Загальноосвітня школа I-III ступенів с.Човнови...,0523186601,"[2018, 2019, 2020, 2021]",05,загальноосвтняшкола3ступсчовновицяоратвськогор...,
991,Загальноосвітня школа I-III ступенів с.Чернявк...,0523186901,"[2018, 2019, 2020, 2021]",05,загальноосвтняшкола3ступсчернявкаоратвськогора...,
995,"Навчально виховний комплекс ""Загальноосвітня ш...",0523255100,"[2016, 2017]",05,навчальновиховнийкомплексзагальноосвтняшкола3с...,
...,...,...,...,...,...,...
27312,Гімназія біотехнологій №177 Солом'янського рай...,8038900000,"[2016, 2017, 2018, 2019, 2020, 2021, 2022]",80,гмназяботехнологй177соломянськогорайонумкив,
27328,Київський спортивний ліцей-інтернат міського п...,8038900000,"[2016, 2017, 2018, 2019, 2020, 2021, 2022]",80,кивськийспортивнийлцейнтернатмськогопдпорядкуван,
27390,Технічний ліцей Національного технічного уніве...,8038900000,"[2016, 2017, 2018, 2019, 2020, 2021, 2022]",80,технчнийлцейнацональноготехнчногоунверситетуук...,
27479,"Товариство з обмеженою відповідальністю ""Франц...",8039100000,"[2016, 2017, 2018, 2019, 2020, 2021, 2022]",80,товариствозобмеженоювдповдальнстюфранцузькийлц...,


In [ ]:
youcontrol[youcontrol.EDRPOU == '26125756']['Назва'].unique()

array([], dtype=object)

In [ ]:
youcontrol[youcontrol.EDRPOU == '26125756'].id.unique()

array([], dtype=object)